In [1]:
    user = User.objects.get(pk=1)
    sent_coupons = GiftCoupon.objects.filter(Q(sender=user) | Q(usergiftcoupon=UserGiftCoupon(prior_user=user)))   
    sent_coupons_values = (
            sent_coupons.annotate(
                prior_username=F('usergiftcoupon__prior_user__username'),
                prior_user=F('usergiftcoupon__prior_user')
            ).values('receiver_name', 'price', 'created_time', 'expire_date',
                     'sender', 'is_used', 'usergiftcoupon__user__username', 'prior_user', 'prior_username'))

(0.002) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 1; args=(1,)


In [3]:
list(sent_coupons_values)

(0.001) SELECT "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."price", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."is_used", T5."username", T4."username" AS "prior_username", "coupon_usergiftcoupon"."prior_user_id" AS "prior_user" FROM "coupon_giftcoupon" LEFT OUTER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") LEFT OUTER JOIN "accounts_user" T4 ON ("coupon_usergiftcoupon"."prior_user_id" = T4."id") LEFT OUTER JOIN "accounts_user" T5 ON ("coupon_usergiftcoupon"."user_id" = T5."id") WHERE ("coupon_giftcoupon"."sender_id" = 1 OR "coupon_usergiftcoupon"."id" IS NULL); args=(1,)


[{'receiver_name': '장영실',
  'price': 50000,
  'created_time': datetime.datetime(2019, 5, 16, 2, 5, 34, 838630, tzinfo=<UTC>),
  'expire_date': datetime.date(2019, 8, 14),
  'sender': 1,
  'is_used': False,
  'usergiftcoupon__user__username': 'sora.son',
  'prior_username': 'sora.son',
  'prior_user': 1},
 {'receiver_name': '정약용',
  'price': 30000,
  'created_time': datetime.datetime(2019, 5, 15, 9, 20, 2, 381653, tzinfo=<UTC>),
  'expire_date': datetime.date(2019, 8, 13),
  'sender': 2,
  'is_used': False,
  'usergiftcoupon__user__username': None,
  'prior_username': None,
  'prior_user': None},
 {'receiver_name': '홍길동',
  'price': 30000,
  'created_time': datetime.datetime(2019, 5, 15, 3, 29, 30, 329862, tzinfo=<UTC>),
  'expire_date': datetime.date(2019, 8, 13),
  'sender': 1,
  'is_used': True,
  'usergiftcoupon__user__username': None,
  'prior_username': None,
  'prior_user': None}]

In [14]:
coupons = []
for coupon in sent_coupons_values:
    is_registered = True
    is_buy = False

    ## sender 가 user 이나 이미 양도된 경우 prior
    sender = coupon['sender']
    prior_user = coupon['prior_user']

    receiver_name = coupon['usergiftcoupon__user__username']
    print(receiver_name)

    if sender == user.id:
        receiver_name = coupon['receiver_name']
        is_buy = True # 구매한 쿠폰
        if not prior_user:
            is_registered = False
    elif prior_user == user.id:
        receiver_name = coupon['usergiftcoupon__user__username']

    coupons += [{
        'receiver_name': receiver_name,
        'price': coupon['price'],
        'create_date': coupon['created_time'].date(),
        'expire_date': coupon['expire_date'],
        'is_registered': is_registered,
        'is_buy': is_buy,
    }]

sora.son
None
None


In [10]:
coupons

[{'receiver_name': '장영실',
  'price': 50000,
  'create_date': datetime.date(2019, 5, 16),
  'expire_date': datetime.date(2019, 8, 14),
  'is_registered': True,
  'is_buy': True},
 {'receiver_name': None,
  'price': 30000,
  'create_date': datetime.date(2019, 5, 15),
  'expire_date': datetime.date(2019, 8, 13),
  'is_registered': True,
  'is_buy': False},
 {'receiver_name': '홍길동',
  'price': 30000,
  'create_date': datetime.date(2019, 5, 15),
  'expire_date': datetime.date(2019, 8, 13),
  'is_registered': False,
  'is_buy': True}]

In [10]:
user = User.objects.get(id=12)
sent_coupons = GiftCoupon.objects.filter(Q(sender=user) | Q(usergiftcoupon=UserGiftCoupon(pr)))

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 12; args=(12,)


In [18]:
sent_coupons = GiftCoupon.objects.filter(Q(sender=user) | Q(usergiftcoupon__prior_user=user))

In [19]:
sent_coupons

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" LEFT OUTER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE ("coupon_giftcoupon"."sender_id" = 12 OR "coupon_usergiftcoupon"."prior_user_id" = 12)  LIMIT 21; args=(12, 12)


<QuerySet []>

In [6]:
SELECT 
"coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" 
FROM "coupon_giftcoupon" 
LEFT OUTER JOIN "coupon_usergiftcoupon" 
ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") 
WHERE ("coupon_giftcoupon"."sender_id" = 12 OR "coupon_usergiftcoupon"."id" IS NULL)  LIMIT 21; args=(12,)

SyntaxError: invalid syntax (<ipython-input-6-9d7c2eca3b4f>, line 2)

In [7]:
UserGiftCoupon(prior_user=user)

RelatedObjectDoesNotExist: UserGiftCoupon has no gift_coupon.

In [14]:
UserGiftCoupon.objects.all()

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant" FROM "coupon_usergiftcoupon"  LIMIT 21; args=()
(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" WHERE "coupon_giftcoupon"."id" = 1; args=(1,)
(0.000) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_t

<QuerySet [<UserGiftCoupon: 7b64b886-f8bd-4154-90a3-df5a8150f8a4>, <UserGiftCoupon: 7e67d492-0578-48b8-a307-41c8a7bbbde0>, <UserGiftCoupon: 0d2bf9c0-6584-49d2-b296-243b3c91e7c8>, <UserGiftCoupon: e61c3d08-e2e1-4842-bbb1-b5be5dbc7125>, <UserGiftCoupon: e61c3d08-e2e1-4842-bbb1-b5be5dbc7125>]>

In [21]:
        gift_coupon_obj = GiftCoupon.objects.filter(usergiftcoupon=2)
        import datetime
        original_expire_date = gift_coupon_obj[0].expire_date
        over_expire_date = original_expire_date-datetime.timedelta(days=1)

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE "coupon_usergiftcoupon"."id" = 2  LIMIT 1; args=(2,)


In [23]:
original_expire_date

datetime.date(2019, 8, 15)

In [24]:
over_expire_date

datetime.date(2019, 8, 14)

In [28]:
gift_coupon_obj = GiftCoupon.objects.filter(usergiftcoupon=12)

In [29]:
original_expire_date = gift_coupon_obj[0].expire_date
over_expire_date = original_expire_date-datetime.timedelta(days=1)
gift_coupon_obj.update(expire_date=over_expire_date)

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE "coupon_usergiftcoupon"."id" = 12  LIMIT 1; args=(12,)
(0.002) UPDATE "coupon_giftcoupon" SET "expire_date" = '2019-08-17'::date WHERE "coupon_giftcoupon"."id" IN (SELECT U0."id" FROM "coupon_giftcoupon" U0 INNER JOIN "coupon_usergiftcoupon" U1 ON (U0."id" = U1."gift_coupon_id") WHERE U1."id" = 12); args=(datetime.date(2019, 8, 17), 12)


1

In [18]:
user = User.objects.get(pk=12)

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 12; args=(12,)


In [10]:
(UserGiftCoupon.objects.select_related('gift_coupon')
    .filter(user=user)
    .filter(is_owner=True)
    .filter(gift_coupon__is_used=False))

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_usergiftcoupon" INNER JOIN "coupon_giftcoupon" ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") WHERE ("coupon_usergiftcoupon"."user_id" = 12 AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_gif

<QuerySet [<UserGiftCoupon: e61c3d08-e2e1-4842-bbb1-b5be5dbc7125>]>

In [37]:
import datetime    
available_coupon_objs = (
        UserGiftCoupon.objects.select_related('gift_coupon')
        .filter(user=user)
        .filter(is_owner=True)
        .filter(gift_coupon__is_used=False)
        .filter(gift_coupon__expire_date__gte=datetime.datetime.now().date()))

available_coupon_list = (
    available_coupon_objs
    .annotate(
        price=F('gift_coupon__price'),
        create_date=F('gift_coupon__created_time'),
        expire_date=F('gift_coupon__expire_date'),
        coupon_code=F('gift_coupon__coupon_code'),
        prior_username=F('prior_user__username'),
    ).values())

In [38]:
list(available_coupon_list)

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_giftcoupon"."price" AS "price", "coupon_giftcoupon"."created_time" AS "create_date", "coupon_giftcoupon"."expire_date" AS "expire_date", "coupon_giftcoupon"."coupon_code" AS "coupon_code", T4."username" AS "prior_username" FROM "coupon_usergiftcoupon" INNER JOIN "coupon_giftcoupon" ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") LEFT OUTER JOIN "accounts_user" T4 ON ("coupon_usergiftcoupon"."prior_user_id" = T4."id") WHERE ("coupon_usergiftcoupon"."user_id" = 12 AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_giftcoupon"."is_used" = false AND "coupon_giftcoupon"."expire_date" >= '2019-05-22'::date); args=(12, True, False, datetim

[{'id': 11,
  'created_time': datetime.datetime(2019, 5, 22, 8, 26, 35, 559341, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 5, 22, 8, 26, 35, 559374, tzinfo=<UTC>),
  'gift_coupon_id': 3,
  'user_id': 12,
  'prior_user_id': 10,
  'is_owner': True,
  'is_registrant': False,
  'price': 50000,
  'create_date': datetime.datetime(2019, 5, 17, 2, 30, 41, 159000, tzinfo=<UTC>),
  'expire_date': datetime.date(2019, 8, 15),
  'coupon_code': UUID('e61c3d08-e2e1-4842-bbb1-b5be5dbc7125'),
  'prior_username': 'sora.son'}]

In [40]:
available_coupons = (
    UserGiftCoupon.objects.select_related('gift_coupon')
    .filter(user=user)
    .filter(is_owner=True)
    .filter(gift_coupon__is_used=False)
    .filter(gift_coupon__expire_date__gte=datetime.datetime.now().date())
    .annotate(
        price=F('gift_coupon__price'),
        create_date=F('gift_coupon__created_time'),
        expire_date=F('gift_coupon__expire_date'),
        coupon_code=F('gift_coupon__coupon_code'),
        prior_username=F('prior_user__username'),
    ).values())

available_coupon_list = list(available_coupons,)

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_giftcoupon"."price" AS "price", "coupon_giftcoupon"."created_time" AS "create_date", "coupon_giftcoupon"."expire_date" AS "expire_date", "coupon_giftcoupon"."coupon_code" AS "coupon_code", T4."username" AS "prior_username" FROM "coupon_usergiftcoupon" INNER JOIN "coupon_giftcoupon" ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") LEFT OUTER JOIN "accounts_user" T4 ON ("coupon_usergiftcoupon"."prior_user_id" = T4."id") WHERE ("coupon_usergiftcoupon"."user_id" = 12 AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_giftcoupon"."is_used" = false AND "coupon_giftcoupon"."expire_date" >= '2019-05-22'::date); args=(12, True, False, datetim

In [41]:
available_coupon_list

[{'id': 11,
  'created_time': datetime.datetime(2019, 5, 22, 8, 26, 35, 559341, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 5, 22, 8, 26, 35, 559374, tzinfo=<UTC>),
  'gift_coupon_id': 3,
  'user_id': 12,
  'prior_user_id': 10,
  'is_owner': True,
  'is_registrant': False,
  'price': 50000,
  'create_date': datetime.datetime(2019, 5, 17, 2, 30, 41, 159000, tzinfo=<UTC>),
  'expire_date': datetime.date(2019, 8, 15),
  'coupon_code': UUID('e61c3d08-e2e1-4842-bbb1-b5be5dbc7125'),
  'prior_username': 'sora.son'}]

In [42]:
UserGiftCoupon.objects.select_related('gift_coupon')
            .filter(user=user)
            .filter(is_owner=True)
            .filter(gift_coupon__is_used=False)
            .filter(gift_coupon__expire_date__gte=datetime.datetime.now().date())
            .annotate(
                price=F('gift_coupon__price'),
                create_date=F('gift_coupon__created_time'),
                expire_date=F('gift_coupon__expire_date'),
                coupon_code=F('gift_coupon__coupon_code'),
                prior_username=F('prior_user__username'),
            ).values()

IndentationError: unexpected indent (<ipython-input-42-52cf04134040>, line 2)

In [43]:
GiftCoupon.objects.filter(usergiftcoupon__is_owner=True).filter(usergiftcoupon__user = user)

(0.002) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") INNER JOIN "coupon_usergiftcoupon" T3 ON ("coupon_giftcoupon"."id" = T3."gift_coupon_id") WHERE ("coupon_usergiftcoupon"."is_owner" = true AND T3."user_id" = 12)  LIMIT 21; args=(True, 12)


<QuerySet [<GiftCoupon: e61c3d08-e2e1-4842-bbb1-b5be5dbc7125>, <GiftCoupon: e61c3d08-e2e1-4842-bbb1-b5be5dbc7125>]>

In [45]:
(GiftCoupon.objects
        .filter(usergiftcoupon__is_owner=True).filter(usergiftcoupon__user = user)
        .filter(is_used=False)
        .filter(expire_date__gte=datetime.datetime.now().date())
        .values())

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") INNER JOIN "coupon_usergiftcoupon" T3 ON ("coupon_giftcoupon"."id" = T3."gift_coupon_id") WHERE ("coupon_usergiftcoupon"."is_owner" = true AND T3."user_id" = 12 AND "coupon_giftcoupon"."is_used" = false AND "coupon_giftcoupon"."expire_date" >= '2019-05-23'::date)  LIMIT 21; args=(True, 12, False, datetime.date(2019, 5, 23))


<QuerySet [{'id': 3, 'created_time': datetime.datetime(2019, 5, 17, 2, 30, 41, 159000, tzinfo=<UTC>), 'updated_time': datetime.datetime(2019, 5, 17, 2, 30, 41, 159000, tzinfo=<UTC>), 'sender_id': 10, 'sender_msg': '구구구구구                    ', 'receiver_name': '이순신', 'receiver_phone': '', 'receiver_email': 'sora.son@deliveryhero.co.kr', 'expire_date': datetime.date(2019, 8, 15), 'coupon_code': UUID('e61c3d08-e2e1-4842-bbb1-b5be5dbc7125'), 'price': 50000, 'is_used': False, 'used_date': None}, {'id': 3, 'created_time': datetime.datetime(2019, 5, 17, 2, 30, 41, 159000, tzinfo=<UTC>), 'updated_time': datetime.datetime(2019, 5, 17, 2, 30, 41, 159000, tzinfo=<UTC>), 'sender_id': 10, 'sender_msg': '구구구구구                    ', 'receiver_name': '이순신', 'receiver_phone': '', 'receiver_email': 'sora.son@deliveryhero.co.kr', 'expire_date': datetime.date(2019, 8, 15), 'coupon_code': UUID('e61c3d08-e2e1-4842-bbb1-b5be5dbc7125'), 'price': 50000, 'is_used': False, 'used_date': None}]>

In [54]:
GiftCoupon.objects.filter(Q(sender=user) | Q(usergiftcoupon__prior_user=user)).\
order_by('-created_time').\
order_by('-usergiftcoupon')

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" LEFT OUTER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE ("coupon_giftcoupon"."sender_id" = 12 OR "coupon_usergiftcoupon"."prior_user_id" = 12) ORDER BY "coupon_usergiftcoupon"."id" DESC  LIMIT 21; args=(12, 12)


<QuerySet []>

In [55]:
SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_giftcoupon"."price" AS "price", "coupon_giftcoupon"."created_time" AS "create_date", "coupon_giftcoupon"."expire_date" AS "expire_date", "coupon_usergiftcoupon"."gift_coupon_id" AS "coupon_id", "coupon_giftcoupon"."coupon_code" AS "coupon_code", T4."username" AS "prior_username" 
FROM "coupon_usergiftcoupon" 
INNER JOIN "coupon_giftcoupon" 
ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") 
LEFT OUTER JOIN "accounts_user" T4 ON ("coupon_usergiftcoupon"."prior_user_id" = T4."id") 
WHERE ("coupon_usergiftcoupon"."user_id" = 10 AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_giftcoupon"."is_used" = false AND "coupon_giftcoupon"."expire_date" >= '2019-05-23'::date); args=(10, True, 

SyntaxError: invalid syntax (<ipython-input-55-17d484efbf3c>, line 1)

In [56]:
user= User.objects.get(username='sora.son')
received_coupons = (UserGiftCoupon.objects.select_related('gift_coupon')
                    .filter(user=user).order_by('-created_time'))

coupons_values = received_coupons.values(
    'id', 'gift_coupon__id', 'gift_coupon__coupon_code', 'gift_coupon__is_used', 'is_owner',
    'prior_user', 'prior_user__username', 'gift_coupon__sender__username', 'gift_coupon__expire_date',
    'gift_coupon__price', 'gift_coupon__created_time', 'created_time',
    )


(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'sora.son'; args=('sora.son',)


In [59]:
import datetime
from datetime import datetime
today=datetime.now()

In [62]:
for coupon in coupons_values:

    expire_date = coupon['gift_coupon__expire_date']
    is_used = coupon['gift_coupon__is_used']

    if today <= expire_date:
        is_expired = False

TypeError: can't compare datetime.datetime to datetime.date

In [63]:
SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" 
FROM "restaurant_restaurant" 
INNER JOIN "accounts_user_restaurant" 
ON ("restaurant_restaurant"."id" = "accounts_user_restaurant"."restaurant_id") 
WHERE ("accounts_user_restaurant"."user_id" = 10 AND "restaurant_restaurant"."id" = 1); args=(10, 1)


SyntaxError: invalid syntax (<ipython-input-63-27fb5b6a152e>, line 1)

In [1]:
user = User.objects.get(username='sora.son')
user.subscribed_restaurants.filter(pk=1)

(0.002) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'sora.son'; args=('sora.son',)
(0.014) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_char

<QuerySet []>

In [3]:
if user.subscribed_restaurants.filter(pk=1):
    subscribe_flag = False
    print(subscribe_flag)
else:
    subscribe_flag = True
    print(subscribe_flag)

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" FROM "restaurant_restaurant" INNER JOIN "accounts_user_subscribed_restaurants" ON ("restaurant_restaurant"."id" = "accounts_user_subscribed_restaurants"."restaurant_id") WHERE ("accounts_user_subscribed_restaurants"."user_id" = 10 AND "restaurant_restaurant"."id" = 1); args=(10, 1)


True


In [10]:

registered_coupon_id =17
gift_coupon = GiftCoupon.objects.filter(
    is_used=False,
    usergiftcoupon=registered_coupon_id,
    usergiftcoupon__is_owner=True,
    usergiftcoupon__user=user
)

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'sora.son'; args=('sora.son',)


In [20]:
user
gift_coupon[0].price

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE ("coupon_giftcoupon"."is_used" = false AND "coupon_usergiftcoupon"."id" = 17 AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_usergiftcoupon"."user_id" = 10)  LIMIT 1; args=(False, 17, True, 10)


10000

In [14]:
gift_coupon.update(is_used = True)

(0.002) UPDATE "coupon_giftcoupon" SET "is_used" = true WHERE "coupon_giftcoupon"."id" IN (SELECT U0."id" FROM "coupon_giftcoupon" U0 INNER JOIN "coupon_usergiftcoupon" U1 ON (U0."id" = U1."gift_coupon_id") WHERE (U0."is_used" = false AND U1."id" = 17 AND U1."is_owner" = true AND U1."user_id" = 10)); args=(True, False, 17, True, 10)


1

In [15]:
gift_coupon.price


AttributeError: 'QuerySet' object has no attribute 'price'

In [28]:
user = User.objects.get(username='sora.son')
coupon_code = (GiftCoupon.objects
                .filter(sender=user)
                .filter(pk=10)[0].coupon_code)

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'sora.son'; args=('sora.son',)
(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftco

In [31]:

gift_coupon=(GiftCoupon.objects
.get(
    usergiftcoupon__is_owner=True,
    usergiftcoupon__user=user,
    coupon_code=coupon_code))

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE ("coupon_giftcoupon"."coupon_code" = '058036c2-91dc-461e-b0c9-54625478ef1d'::uuid AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_usergiftcoupon"."user_id" = 10); args=(UUID('058036c2-91dc-461e-b0c9-54625478ef1d'), True, 10)


In [32]:
gift_coupon.id

10

In [33]:
def register_coupon():
    gift_coupon=(GiftCoupon.objects
    .get(
        usergiftcoupon__is_owner=True,
        usergiftcoupon__user=user,
        coupon_code=coupon_code))
    return gift_coupon

In [35]:
gift_coupon=register_coupon()

(0.001) SELECT "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_giftcoupon" INNER JOIN "coupon_usergiftcoupon" ON ("coupon_giftcoupon"."id" = "coupon_usergiftcoupon"."gift_coupon_id") WHERE ("coupon_giftcoupon"."coupon_code" = '058036c2-91dc-461e-b0c9-54625478ef1d'::uuid AND "coupon_usergiftcoupon"."is_owner" = true AND "coupon_usergiftcoupon"."user_id" = 10); args=(UUID('058036c2-91dc-461e-b0c9-54625478ef1d'), True, 10)


In [36]:
gift_coupon.id

10

In [37]:
Restaurant.objects.filter(subscribers__in=[user])

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" FROM "restaurant_restaurant" INNER JOIN "accounts_user_subscribed_restaurants" ON ("restaurant_restaurant"."id" = "accounts_user_subscribed_restaurants"."restaurant_id") WHERE "accounts_user_subscribed_restaurants"."user_id" IN (10)  LIMIT 21; args=(10,)


<QuerySet []>

In [38]:
user.subscribed_restaurants.all()

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" FROM "restaurant_restaurant" INNER JOIN "accounts_user_subscribed_restaurants" ON ("restaurant_restaurant"."id" = "accounts_user_subscribed_restaurants"."restaurant_id") WHERE "accounts_user_subscribed_restaurants"."user_id" = 10  LIMIT 21; args=(10,)


<QuerySet []>

In [11]:
user = User.objects.get(id=12)
registered_coupon = (UserGiftCoupon.objects.select_related('gift_coupon')
                                 .filter(user=user)
                                 .filter(is_checked=False)
                                 .filter(is_owner=True).filter(gift_coupon__is_used=False))

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 12; args=(12,)


In [7]:
registered_coupon

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_usergiftcoupon"."is_checked", "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_usergiftcoupon" INNER JOIN "coupon_giftcoupon" ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") WHERE ("coupon_usergiftcoupon"."user_id" = 10 AND "coupon_usergiftcou

<QuerySet [<UserGiftCoupon: e61c3d08-e2e1-4842-bbb1-b5be5dbc7125>, <UserGiftCoupon: 6383a462-2928-4447-b5dc-b7b2ee8925ab>, <UserGiftCoupon: 058036c2-91dc-461e-b0c9-54625478ef1d>]>

In [20]:
unchecked_handover_coupon_list = (
    UserGiftCoupon.objects.select_related('gift_coupon')
    .filter(
        user=user,
        is_checked=False,
        is_owner=True,
        is_registrant=False)

)

In [21]:
unchecked_handover_coupon_list

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_usergiftcoupon"."is_checked", "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_usergiftcoupon" INNER JOIN "coupon_giftcoupon" ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") WHERE ("coupon_usergiftcoupon"."is_checked" = false AND "coupon_userg

<QuerySet [<UserGiftCoupon: 7e67d492-0578-48b8-a307-41c8a7bbbde0>]>

In [28]:
user = User.objects.get(id=12)
unchecked_handover_coupon = (
            UserGiftCoupon.objects.select_related('gift_coupon')
            .filter(
                user=user,
                is_checked=False,
                is_owner=True,
                is_registrant=False))

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 12; args=(12,)


In [29]:
if not unchecked_handover_coupon:
    print('없음')
else:
    unchecked_handover_coupon.update(is_checked=True)

(0.001) SELECT "coupon_usergiftcoupon"."id", "coupon_usergiftcoupon"."created_time", "coupon_usergiftcoupon"."updated_time", "coupon_usergiftcoupon"."gift_coupon_id", "coupon_usergiftcoupon"."user_id", "coupon_usergiftcoupon"."prior_user_id", "coupon_usergiftcoupon"."is_owner", "coupon_usergiftcoupon"."is_registrant", "coupon_usergiftcoupon"."is_checked", "coupon_giftcoupon"."id", "coupon_giftcoupon"."created_time", "coupon_giftcoupon"."updated_time", "coupon_giftcoupon"."sender_id", "coupon_giftcoupon"."sender_msg", "coupon_giftcoupon"."receiver_name", "coupon_giftcoupon"."receiver_phone", "coupon_giftcoupon"."receiver_email", "coupon_giftcoupon"."expire_date", "coupon_giftcoupon"."coupon_code", "coupon_giftcoupon"."price", "coupon_giftcoupon"."is_used", "coupon_giftcoupon"."used_date" FROM "coupon_usergiftcoupon" INNER JOIN "coupon_giftcoupon" ON ("coupon_usergiftcoupon"."gift_coupon_id" = "coupon_giftcoupon"."id") WHERE ("coupon_usergiftcoupon"."is_checked" = false AND "coupon_userg

없음


In [31]:
logged_in_user =User.objects.all()[0]
order_history_list = list(Order.objects.history_list(logged_in_user))
cart_items_list = list(CartItem.objects.filtering_user(logged_in_user))

(0.007) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user"  LIMIT 1; args=()


AttributeError: 'Manager' object has no attribute 'history_list'

In [3]:
import enum
class DeliveryStatus(enum.IntEnum):
    WATING = 0
    DELIVERING = 1
    COMPLETE = 2
from datetime import datetime
today = datetime.now().date()
one_hour = 1
current_hour = datetime.now().hour
created_hour = current_hour
user_address = '서초1동'

order = (
    CartItem.objects
        .select_related('cart', 'menu')
        .values(
        address=F('cart__order__address'),
        delivery_completed=F('cart__order__delivery_status'),
        created_time=F('cart__order__created_time'),
    )
        .filter(
        address__contains=user_address,
        delivery_completed=DeliveryStatus.COMPLETE,
        created_time__hour=created_hour,
    )
        .values(
        'menu__name',
    )
        .annotate(
        menu=F('menu__name'),
        menu_quantity=Sum('quantity')
    )
        .values(
        'menu',
        'menu_quantity',
        'menu_id',
        'menu__img',
    )
        .order_by('-menu_quantity')[:1]
)
order
order[0]  # check_order_for_except_or_not
        

(0.002) SELECT "cart_cartitem"."menu_id", "menu_menu"."img", "menu_menu"."name" AS "menu", SUM("cart_cartitem"."quantity") AS "menu_quantity" FROM "cart_cartitem" INNER JOIN "cart_cart" ON ("cart_cartitem"."cart_id" = "cart_cart"."id") LEFT OUTER JOIN "order_order" ON ("cart_cart"."id" = "order_order"."cart_id") INNER JOIN "menu_menu" ON ("cart_cartitem"."menu_id" = "menu_menu"."id") WHERE ("order_order"."address"::text LIKE '%서초1동%' AND EXTRACT('hour' FROM "order_order"."created_time" AT TIME ZONE 'Asia/Seoul') = 18 AND "order_order"."delivery_status" = 2) GROUP BY "menu_menu"."name", "cart_cartitem"."menu_id", "menu_menu"."img" ORDER BY "menu_quantity" DESC  LIMIT 1; args=('%서초1동%', 18, 2)


{'menu_id': 2,
 'menu__img': 'menu/2019/05/01/갈비천왕.jpg',
 'menu': '갈비천왕',
 'menu_quantity': 67}

In [16]:
from restaurant.models import Restaurant
#list(Restaurant.objects.all().values())
list(Restaurant.objects.filter(pk=13).values('tel', 'min_order_price', 'order_way', 'origin', 'delivery_charge', 'info',
                                            'event', 'event_img', 'operation_start_hour', 'operation_end_hour'))[0]

(0.001) SELECT "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."event", "restaurant_restaurant"."event_img", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" FROM "restaurant_restaurant" WHERE "restaurant_restaurant"."id" = 13; args=(13,)


{'tel': '1',
 'min_order_price': 0,
 'order_way': '1',
 'origin': '12',
 'delivery_charge': 0,
 'info': '2',
 'event': '3',
 'event_img': '',
 'operation_start_hour': datetime.time(19, 44, 2),
 'operation_end_hour': datetime.time(19, 44, 3)}

In [18]:
prior_restaurant_obj = Restaurant.objects.filter(pk=13).values()
prior_restaurant_dict = list(prior_restaurant_obj)[0]
prior_restaurant_dict['operation_start_hour']

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."event", "restaurant_restaurant"."event_img", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" FROM "restaurant_restaurant" WHERE "restaurant_restaurant"."id" = 13; args=(13,)


datetime.time(10, 44, 2)

In [21]:
'-'.join([str(prior_restaurant_dict['operation_start_hour']), str(prior_restaurant_dict['operation_end_hour'])])
           

TypeError: sequence item 0: expected str instance, datetime.time found

In [38]:
user=User.objects.get(pk=10)

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 10; args=(10,)


In [42]:
subscribed_restaurants_qs
=user.subscribed_restaurants.all()

In [39]:
subscribed_restaurants_qs

<User: sora.son>

In [43]:
from restaurant.models import RestaurantTimeline

In [46]:
RestaurantTimeline.objects.prefetch_related('restaurant')


(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_notice", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info" FROM "restaurant_restauranttimeline"  LIMIT 21; args=()
(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."event", "restauran

<QuerySet [<RestaurantTimeline: RestaurantTimeline object (6)>, <RestaurantTimeline: RestaurantTimeline object (7)>, <RestaurantTimeline: RestaurantTimeline object (8)>, <RestaurantTimeline: RestaurantTimeline object (9)>, <RestaurantTimeline: RestaurantTimeline object (10)>, <RestaurantTimeline: RestaurantTimeline object (11)>, <RestaurantTimeline: RestaurantTimeline object (12)>, <RestaurantTimeline: RestaurantTimeline object (16)>, <RestaurantTimeline: RestaurantTimeline object (17)>, <RestaurantTimeline: RestaurantTimeline object (18)>, <RestaurantTimeline: RestaurantTimeline object (19)>, <RestaurantTimeline: RestaurantTimeline object (20)>, <RestaurantTimeline: RestaurantTimeline object (21)>, <RestaurantTimeline: RestaurantTimeline object (22)>, <RestaurantTimeline: RestaurantTimeline object (23)>, <RestaurantTimeline: RestaurantTimeline object (24)>, <RestaurantTimeline: RestaurantTimeline object (25)>, <RestaurantTimeline: RestaurantTimeline object (26)>]>

In [52]:
list(RestaurantTimeline.objects.select_related('restaurant').all().values())

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_notice", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info" FROM "restaurant_restauranttimeline"; args=()


[{'id': 6,
  'created_time': datetime.datetime(2019, 6, 3, 13, 11, 22, 286459, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 3, 13, 11, 22, 286485, tzinfo=<UTC>),
  'restaurant_id': 13,
  'status': 2,
  'changed_notice': '이벤트',
  'changed_img': 'restaurant/event/2019/06/03/후라이드치킨.jpeg',
  'prior_info': '',
  'post_info': 'event'},
 {'id': 7,
  'created_time': datetime.datetime(2019, 6, 3, 13, 12, 22, 780121, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 3, 13, 12, 22, 780145, tzinfo=<UTC>),
  'restaurant_id': 13,
  'status': 2,
  'changed_notice': '',
  'changed_img': '',
  'prior_info': '',
  'post_info': '10:44:02-20:44:03'},
 {'id': 8,
  'created_time': datetime.datetime(2019, 6, 3, 13, 13, 12, 259357, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 3, 13, 13, 12, 259414, tzinfo=<UTC>),
  'restaurant_id': 13,
  'status': 2,
  'changed_notice': '',
  'changed_img': '',
  'prior_info': '',
  'post_info': '08:44:02-12:44:03'},
 {'id': 9,
 

In [73]:
from timeline.models import TYPE_CHOICES
dict(TYPE_CHOICES)['1']


KeyError: '1'

In [79]:
user_res=User.objects.get(pk=10).subscribed_restaurants

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 10; args=(10,)


In [87]:
User.objects.get(pk=10).subscribed_restaurants.prefetch_related('restauranttimeline_set')

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 10; args=(10,)
(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_resta

<QuerySet [<Restaurant: 교촌치킨>, <Restaurant: 1>]>

In [93]:
subscribed_restaurants_list=User.objects.get(pk=10).subscribed_restaurants.values('id')

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 10; args=(10,)


In [98]:
subscribed_restaurants_ids=[restaurant['id'] for restaurant in subscribed_restaurants_list]

In [100]:
subscribed_restaurants_ids

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 10; args=(10,)


AttributeError: 'User' object has no attribute 'subscribed_restaurants_ids'

In [101]:
user=User.objects.get(pk=10)
subscribed_restaurants = user.subscribed_restaurants.values('id')
subscribed_restaurants_ids = [restaurant['id'] for restaurant in subscribed_restaurants]

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 10; args=(10,)
(0.001) SELECT "restaurant_restaurant"."id" FROM "restaurant_restaurant" INNER JOIN "accounts_user_subscribed_restaurants" ON ("restaurant_restaurant"."id" = "accounts_user_subscribed_restaurants"."restaurant_id") WHERE "accounts_user_subscribed_restaurants"."user_id" = 10; args=(10,)


In [102]:
subscribed_restaurants_ids

[8, 12]

In [105]:
from restaurant.models import Restaurant
Restuarant.objects.all()[0]

NameError: name 'Restuarant' is not defined

In [106]:
from restaurant.models import Restaurant

In [110]:
Restaurant.objects.all()[0].id

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."event", "restaurant_restaurant"."event_img", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour" FROM "restaurant_restaurant"  LIMIT 1; args=()


1

In [10]:
user =User.objects.get(pk=9)
RestaurantTimeline.objects.filter(pk=3).add(like=user)

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."id" = 9; args=(9,)


AttributeError: 'QuerySet' object has no attribute 'add'

In [1]:
from restaurant.models import Restaurant, RestaurantTimeline
available_restaurant = Restaurant.objects.filter(is_group_purchase=True).order_by('id')

In [23]:
available_restaurant = (
    Restaurant.objects.select_related('menu')
    .filter(
        menu__is_group_purchase=True,
        is_group_purchase=True
    ).order_by('id')
    .annotate(
        menu_id=F('menu'),
        menu_name=F('menu__name'),
        menu_detail=F('menu__detail'),
        menu_price=F('menu__price'),
        max_quantity=F('menu__max_quantity'),
        min_quantity=F('menu__min_quantity'),
    ).values('id', 'title', 'name', 'menu_id', 'menu_name', 'menu_detail', 'menu_price', 'max_quantity', 'min_quantity'))

In [24]:
res_list = list(available_restaurant)

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."title", "restaurant_restaurant"."name", "menu_menu"."id" AS "menu_id", "menu_menu"."name" AS "menu_name", "menu_menu"."detail" AS "menu_detail", "menu_menu"."price" AS "menu_price", "menu_menu"."max_quantity" AS "max_quantity", "menu_menu"."min_quantity" AS "min_quantity" FROM "restaurant_restaurant" INNER JOIN "menu_menu" ON ("restaurant_restaurant"."id" = "menu_menu"."restaurant_id") WHERE ("restaurant_restaurant"."is_group_purchase" = true AND "menu_menu"."is_group_purchase" = true) ORDER BY "restaurant_restaurant"."id" ASC; args=(True, True)


In [29]:
res_list

for res in res_list:
    print(res['id'])

8
8
8
8
8
9
9
9
9


In [45]:
available_menus = (
    Menu.objects.select_related('restaurant')
    .filter(
        is_group_purchase=True,
        restaurant__is_group_purchase=True
    ).order_by('restaurant_id')
    .annotate(
        restaurant_name = F('restaurant__name'),
        restaurant_title = F('restaurant__title'),
    ).values('restaurant_id', 'restaurant_name', 'restaurant_title',
             'id', 'name', 'detail', 'price', 'min_quantity', 'max_quantity')
)

In [46]:
list(available_menus)

(0.001) SELECT "menu_menu"."restaurant_id", "menu_menu"."id", "menu_menu"."name", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."min_quantity", "menu_menu"."max_quantity", "restaurant_restaurant"."name" AS "restaurant_name", "restaurant_restaurant"."title" AS "restaurant_title" FROM "menu_menu" INNER JOIN "restaurant_restaurant" ON ("menu_menu"."restaurant_id" = "restaurant_restaurant"."id") WHERE ("menu_menu"."is_group_purchase" = true AND "restaurant_restaurant"."is_group_purchase" = true) ORDER BY "menu_menu"."restaurant_id" ASC; args=(True, True)


[{'restaurant_id': 8,
  'id': 28,
  'name': '교촌 라이스 세트',
  'detail': '쌀로 만든 궁극의 바사함, 교촌라이스세트!',
  'price': 21000,
  'min_quantity': 0,
  'max_quantity': 0,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 8,
  'id': 29,
  'name': '교촌 오리지날',
  'detail': '교촌만의 차별화된 마늘과 간장소스의 풍부한 맛이 어우러진 한 마리 치킨',
  'price': 15000,
  'min_quantity': 0,
  'max_quantity': 0,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 8,
  'id': 30,
  'name': '교촌콤보',
  'detail': '마늘과 간장소스의 풍부한 맛에 가장 인기있는 부위인 날개와 다리를 함께 즐길 수 있는 메뉴',
  'price': 17000,
  'min_quantity': 0,
  'max_quantity': 0,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 8,
  'id': 31,
  'name': '교촌레드스틱',
  'detail': '국내산 청양 홍고추의 매콤함에 담백한 다리의 맛이 어우러진 치킨',
  'price': 18000,
  'min_quantity': 0,
  'max_quantity': 0,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 8,
  'id': 32,
  'name': '교촌레드콤보',
  'detail': '국내산 

In [81]:
restaurants_list = []
menu_list = []
restaurant_id
for menu in available_menus:
    if not restaurants_list:
        menu_list.append(menu)
        
        
'''        
    menu_list.append(menu)
    if restaurant_id != menu['restaurant_id']:
        restaurant_id = menu['restaurant_id']
        restaurants_list.append(
            {
                'restaurant_id': restaurant_id,
                'restaurant_name': menu['restaurant_name'],
                'restaurant_title': menu['restaurant_title'],
                'menus':menu_list
            })
        menu_list.clear()
'''

In [83]:
available_restaurant = Restaurant.objects.filter(is_group_purchase=True).order_by('id')

In [85]:
available_menus = (
    Menu.objects.select_related('restaurant')
    .filter(
        is_group_purchase=True,
        restaurant__is_group_purchase=True
    ).order_by('restaurant_id')
    .annotate(
        restaurant_name=F('restaurant__name'),
        restaurant_title=F('restaurant__title'),
    ).values('restaurant_id', 'restaurant_name', 'restaurant_title',
             'id', 'name', 'detail', 'price', 'min_quantity', 'max_quantity')
)

In [88]:
available_restuarants = list(available_menus.distinct('restaurant_id').values('restaurant_id'))

(0.001) SELECT DISTINCT ON ("menu_menu"."restaurant_id") "menu_menu"."restaurant_id" FROM "menu_menu" INNER JOIN "restaurant_restaurant" ON ("menu_menu"."restaurant_id" = "restaurant_restaurant"."id") WHERE ("menu_menu"."is_group_purchase" = true AND "restaurant_restaurant"."is_group_purchase" = true) ORDER BY "menu_menu"."restaurant_id" ASC; args=(True, True)


In [89]:
available_restuarants

[{'restaurant_id': 8}, {'restaurant_id': 9}]

In [90]:
available_restaurant = Restaurant.objects.filter(is_group_purchase=True).order_by('id')


In [91]:
available_restaurant.distinct('id')

(0.001) SELECT DISTINCT ON ("restaurant_restaurant"."id") "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."event", "restaurant_restaurant"."event_img", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour", "restaurant_restaurant"."is_group_purchase" FROM "restaurant_restaurant" WHERE "restaurant_restaurant"."is_group_purchase" = true ORDER BY "restaurant_restaurant"."id" ASC  LIMIT 21; args=(True,)


<QuerySet [<Restaurant: 교촌치킨>, <Restaurant: 신전 떡볶이>]>

In [92]:
restaurant_list = (available_restaurant
            .filter(
                menu__is_group_purchase=True,
            ).order_by('id')
            .annotate(
                menu_id=F('menu'),
                menu_name=F('menu__name'),
                menu_detail=F('menu__detail'),
                menu_price=F('menu__price'),
                max_quantity=F('menu__max_quantity'),
                min_quantity=F('menu__min_quantity'),
            ).values('id', 'title', 'name',
                     'menu_id', 'menu_name', 'menu_detail', 'menu_price', 'max_quantity', 'min_quantity'))

In [93]:
restaurant_list

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."title", "restaurant_restaurant"."name", "menu_menu"."id" AS "menu_id", "menu_menu"."name" AS "menu_name", "menu_menu"."detail" AS "menu_detail", "menu_menu"."price" AS "menu_price", "menu_menu"."max_quantity" AS "max_quantity", "menu_menu"."min_quantity" AS "min_quantity" FROM "restaurant_restaurant" INNER JOIN "menu_menu" ON ("restaurant_restaurant"."id" = "menu_menu"."restaurant_id") WHERE ("restaurant_restaurant"."is_group_purchase" = true AND "menu_menu"."is_group_purchase" = true) ORDER BY "restaurant_restaurant"."id" ASC  LIMIT 21; args=(True, True)


<QuerySet [{'id': 8, 'title': '교촌치킨 한양대점', 'name': '교촌치킨', 'menu_id': 28, 'menu_name': '교촌 라이스 세트', 'menu_detail': '쌀로 만든 궁극의 바사함, 교촌라이스세트!', 'menu_price': 21000, 'max_quantity': 0, 'min_quantity': 0}, {'id': 8, 'title': '교촌치킨 한양대점', 'name': '교촌치킨', 'menu_id': 29, 'menu_name': '교촌 오리지날', 'menu_detail': '교촌만의 차별화된 마늘과 간장소스의 풍부한 맛이 어우러진 한 마리 치킨', 'menu_price': 15000, 'max_quantity': 0, 'min_quantity': 0}, {'id': 8, 'title': '교촌치킨 한양대점', 'name': '교촌치킨', 'menu_id': 30, 'menu_name': '교촌콤보', 'menu_detail': '마늘과 간장소스의 풍부한 맛에 가장 인기있는 부위인 날개와 다리를 함께 즐길 수 있는 메뉴', 'menu_price': 17000, 'max_quantity': 0, 'min_quantity': 0}, {'id': 8, 'title': '교촌치킨 한양대점', 'name': '교촌치킨', 'menu_id': 31, 'menu_name': '교촌레드스틱', 'menu_detail': '국내산 청양 홍고추의 매콤함에 담백한 다리의 맛이 어우러진 치킨', 'menu_price': 18000, 'max_quantity': 0, 'min_quantity': 0}, {'id': 8, 'title': '교촌치킨 한양대점', 'name': '교촌치킨', 'menu_id': 32, 'menu_name': '교촌레드콤보', 'menu_detail': '국내산 청양 홍고추의 매콤한 맛에 날개와 다리를 함께 즐길 수 있는 메뉴', 'menu_price': 18000, 'max_quantity'

In [95]:
available_menus = (
            Menu.objects.select_related('restaurant')
            .filter(
                is_group_purchase=True,
                restaurant__is_group_purchase=True
            ).order_by('restaurant_id')
            .annotate(
                restaurant_name=F('restaurant__name'),
                restaurant_title=F('restaurant__title'),
            ).values('restaurant_id', 'restaurant_name', 'restaurant_title',
                     'id', 'name', 'detail', 'price', 'min_quantity', 'max_quantity')
        )

available_restuarants = list(available_menus.distinct('restaurant_id').values('restaurant_id'))

(0.001) SELECT DISTINCT ON ("menu_menu"."restaurant_id") "menu_menu"."restaurant_id" FROM "menu_menu" INNER JOIN "restaurant_restaurant" ON ("menu_menu"."restaurant_id" = "restaurant_restaurant"."id") WHERE ("menu_menu"."is_group_purchase" = true AND "restaurant_restaurant"."is_group_purchase" = true) ORDER BY "menu_menu"."restaurant_id" ASC; args=(True, True)


In [98]:
menu_list=list(available_menus)

In [107]:
for menu in available_menus:
    print(menu['restaurant_id'])

8
8
8
8
8
9
9
9
9


In [136]:
resturant_ids = set([menu['restaurant_id'] for menu in available_menus])

In [137]:
resturant_ids

{8, 9}

In [153]:
restaurants_list = []
for resturant_id in resturant_ids:
    menu_list = []
    for menu in available_menus:
        if menu['restaurant_id']==resturant_id:
            menu_list.append(menu)
     
    restaurants_list.append({
        'restaurant_id': resturant_id,
        'restaurant_name':'',
        'restaurant_title':'',
        'menus': menu_list
    })
          
            
    

In [151]:
for restaurant in restaurants_list:
    print(restaurant)
    print()

{'restaurant_id': 8, 'restaurant_name': '', 'restaurant_title': '', 'menus': [{'restaurant_id': 8, 'id': 28, 'name': '교촌 라이스 세트', 'detail': '쌀로 만든 궁극의 바사함, 교촌라이스세트!', 'price': 21000, 'min_quantity': 0, 'max_quantity': 0, 'restaurant_name': '교촌치킨', 'restaurant_title': '교촌치킨 한양대점'}, {'restaurant_id': 8, 'id': 29, 'name': '교촌 오리지날', 'detail': '교촌만의 차별화된 마늘과 간장소스의 풍부한 맛이 어우러진 한 마리 치킨', 'price': 15000, 'min_quantity': 0, 'max_quantity': 0, 'restaurant_name': '교촌치킨', 'restaurant_title': '교촌치킨 한양대점'}, {'restaurant_id': 8, 'id': 30, 'name': '교촌콤보', 'detail': '마늘과 간장소스의 풍부한 맛에 가장 인기있는 부위인 날개와 다리를 함께 즐길 수 있는 메뉴', 'price': 17000, 'min_quantity': 0, 'max_quantity': 0, 'restaurant_name': '교촌치킨', 'restaurant_title': '교촌치킨 한양대점'}, {'restaurant_id': 8, 'id': 31, 'name': '교촌레드스틱', 'detail': '국내산 청양 홍고추의 매콤함에 담백한 다리의 맛이 어우러진 치킨', 'price': 18000, 'min_quantity': 0, 'max_quantity': 0, 'restaurant_name': '교촌치킨', 'restaurant_title': '교촌치킨 한양대점'}, {'restaurant_id': 8, 'id': 32, 'name': '교촌레드콤보', 'detail': '국

In [154]:
restaurants_list

[{'restaurant_id': 8,
  'restaurant_name': '',
  'restaurant_title': '',
  'menus': [{'restaurant_id': 8,
    'id': 28,
    'name': '교촌 라이스 세트',
    'detail': '쌀로 만든 궁극의 바사함, 교촌라이스세트!',
    'price': 21000,
    'min_quantity': 0,
    'max_quantity': 0,
    'restaurant_name': '교촌치킨',
    'restaurant_title': '교촌치킨 한양대점'},
   {'restaurant_id': 8,
    'id': 29,
    'name': '교촌 오리지날',
    'detail': '교촌만의 차별화된 마늘과 간장소스의 풍부한 맛이 어우러진 한 마리 치킨',
    'price': 15000,
    'min_quantity': 0,
    'max_quantity': 0,
    'restaurant_name': '교촌치킨',
    'restaurant_title': '교촌치킨 한양대점'},
   {'restaurant_id': 8,
    'id': 30,
    'name': '교촌콤보',
    'detail': '마늘과 간장소스의 풍부한 맛에 가장 인기있는 부위인 날개와 다리를 함께 즐길 수 있는 메뉴',
    'price': 17000,
    'min_quantity': 0,
    'max_quantity': 0,
    'restaurant_name': '교촌치킨',
    'restaurant_title': '교촌치킨 한양대점'},
   {'restaurant_id': 8,
    'id': 31,
    'name': '교촌레드스틱',
    'detail': '국내산 청양 홍고추의 매콤함에 담백한 다리의 맛이 어우러진 치킨',
    'price': 18000,
    'min_quantity': 0,
    'max_

In [156]:
from grouppurchase.models import GroupPurchase, GrouppurchaseStatus
GroupPurchase.objects.filter(restaurant=3, status=not GrouppurchaseStatus.COMPLETE)

(0.002) SELECT "grouppurchase_grouppurchase"."id", "grouppurchase_grouppurchase"."created_time", "grouppurchase_grouppurchase"."updated_time", "grouppurchase_grouppurchase"."restaurant_id", "grouppurchase_grouppurchase"."menu_id", "grouppurchase_grouppurchase"."user_id", "grouppurchase_grouppurchase"."deadline", "grouppurchase_grouppurchase"."target_number", "grouppurchase_grouppurchase"."status", "grouppurchase_grouppurchase"."notice" FROM "grouppurchase_grouppurchase" WHERE ("grouppurchase_grouppurchase"."restaurant_id" = 3 AND "grouppurchase_grouppurchase"."status" = 0)  LIMIT 21; args=(3, 0)


<QuerySet []>

In [160]:
from menu.models import Menu
Menu.objects.get(id=50, is_group_purchase=True)

(0.001) SELECT "menu_menu"."id", "menu_menu"."created_time", "menu_menu"."updated_time", "menu_menu"."restaurant_id", "menu_menu"."name", "menu_menu"."img", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."type", "menu_menu"."hit", "menu_menu"."score", "menu_menu"."is_recommended", "menu_menu"."is_group_purchase", "menu_menu"."min_quantity", "menu_menu"."max_quantity" FROM "menu_menu" WHERE ("menu_menu"."id" = 50 AND "menu_menu"."is_group_purchase" = true); args=(50, True)


DoesNotExist: Menu matching query does not exist.

In [161]:
GroupPurchase.objects.filter(menu=1, status=not GrouppurchaseStatus.COMPLETE)

(0.001) SELECT "grouppurchase_grouppurchase"."id", "grouppurchase_grouppurchase"."created_time", "grouppurchase_grouppurchase"."updated_time", "grouppurchase_grouppurchase"."restaurant_id", "grouppurchase_grouppurchase"."menu_id", "grouppurchase_grouppurchase"."user_id", "grouppurchase_grouppurchase"."deadline", "grouppurchase_grouppurchase"."target_number", "grouppurchase_grouppurchase"."status", "grouppurchase_grouppurchase"."notice" FROM "grouppurchase_grouppurchase" WHERE ("grouppurchase_grouppurchase"."menu_id" = 1 AND "grouppurchase_grouppurchase"."status" = 0)  LIMIT 21; args=(1, 0)


<QuerySet []>

In [167]:
GroupPurchase.objects.filter(menu=28, status= not GrouppurchaseStatus.COMPLETE)

(0.001) SELECT "grouppurchase_grouppurchase"."id", "grouppurchase_grouppurchase"."created_time", "grouppurchase_grouppurchase"."updated_time", "grouppurchase_grouppurchase"."restaurant_id", "grouppurchase_grouppurchase"."menu_id", "grouppurchase_grouppurchase"."user_id", "grouppurchase_grouppurchase"."deadline", "grouppurchase_grouppurchase"."target_number", "grouppurchase_grouppurchase"."status", "grouppurchase_grouppurchase"."notice" FROM "grouppurchase_grouppurchase" WHERE ("grouppurchase_grouppurchase"."menu_id" = 28 AND "grouppurchase_grouppurchase"."status" = 0)  LIMIT 21; args=(28, 0)


<QuerySet []>

In [24]:
from menu.models import Menu
from django.db.models import F, Q
\
```
available_menus = (
            Menu.objects.select_related('restaurant', 'grouppurchase')
            .filter(
                is_group_purchase=True,
                restaurant__is_group_purchase=True,
            ).filter(Q(grouppurchase=None)|Q(grouppurchase__status =3)).order_by('restaurant_id')
            .annotate(
                restaurant_name=F('restaurant__name'),
                restaurant_title=F('restaurant__title'),
            ).values('restaurant_id', 'restaurant_name', 'restaurant_title',
                     'id', 'name', 'detail', 'price', 'min_quantity', 'max_quantity')
        )
```

In [25]:
list(available_menus)

(0.001) SELECT "menu_menu"."restaurant_id", "menu_menu"."id", "menu_menu"."name", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."min_quantity", "menu_menu"."max_quantity", "restaurant_restaurant"."name" AS "restaurant_name", "restaurant_restaurant"."title" AS "restaurant_title" FROM "menu_menu" INNER JOIN "restaurant_restaurant" ON ("menu_menu"."restaurant_id" = "restaurant_restaurant"."id") LEFT OUTER JOIN "grouppurchase_grouppurchase" ON ("menu_menu"."id" = "grouppurchase_grouppurchase"."menu_id") WHERE ("menu_menu"."is_group_purchase" = true AND "restaurant_restaurant"."is_group_purchase" = true AND ("grouppurchase_grouppurchase"."id" IS NULL OR "grouppurchase_grouppurchase"."status" = 3)) ORDER BY "menu_menu"."restaurant_id" ASC; args=(True, True, 3)


[{'restaurant_id': 8,
  'id': 32,
  'name': '교촌레드콤보',
  'detail': '국내산 청양 홍고추의 매콤한 맛에 날개와 다리를 함께 즐길 수 있는 메뉴',
  'price': 18000,
  'min_quantity': 2,
  'max_quantity': 10,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 8,
  'id': 31,
  'name': '교촌레드스틱',
  'detail': '국내산 청양 홍고추의 매콤함에 담백한 다리의 맛이 어우러진 치킨',
  'price': 18000,
  'min_quantity': 2,
  'max_quantity': 10,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 8,
  'id': 30,
  'name': '교촌콤보',
  'detail': '마늘과 간장소스의 풍부한 맛에 가장 인기있는 부위인 날개와 다리를 함께 즐길 수 있는 메뉴',
  'price': 17000,
  'min_quantity': 2,
  'max_quantity': 10,
  'restaurant_name': '교촌치킨',
  'restaurant_title': '교촌치킨 한양대점'},
 {'restaurant_id': 9,
  'id': 33,
  'name': '떡볶이',
  'detail': '맛있는 떡볶이',
  'price': 15000,
  'min_quantity': 2,
  'max_quantity': 10,
  'restaurant_name': '신전 떡볶이',
  'restaurant_title': '신전 떡볶이 한양대점'},
 {'restaurant_id': 9,
  'id': 35,
  'name': '신전치즈김밥',
  'detail': '맛있는 신전치즈김밥',
  'pr

In [28]:
user = User.objects.get(username='admin')
subscribed_restaurants = user.subscribed_restaurants.values('id')



subscribed_restaurants_ids = [restaurant['id'] for restaurant in subscribed_restaurants]

timeline_qs = (
    RestaurantTimeline.objects
    .select_related('restaurant')
    .filter(restaurant_id__in=subscribed_restaurants_ids).order_by('-created_time'))

timeline_values = (
    timeline_qs
    .annotate(
        restaurant_title=F('restaurant__title'),
        restaurant_name=F('restaurant__name'),
        restaurant_img=F('restaurant__img'),
        min_price=F('restaurant__min_order_price'),
        delivery_charge=F('restaurant__delivery_charge'),
        like_count=Count('like')).values())


(0.004) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)
(0.001) SELECT "restaurant_restaurant"."id" FROM "restaurant_restaurant" INNER JOIN "accounts_user_subscribed_restaurants" ON ("restaurant_restaurant"."id" = "accounts_user_subscribed_restaurants"."restaurant_id") WHERE "accounts_user_subscribed_restaurants"."user_id" = 1; args=(1,)


In [30]:
list(timeline_values)

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_data", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info", "restaurant_restauranttimeline"."timeline_type", "restaurant_restauranttimeline"."changed_field", "restaurant_restaurant"."title" AS "restaurant_title", "restaurant_restaurant"."name" AS "restaurant_name", "restaurant_restaurant"."img" AS "restaurant_img", "restaurant_restaurant"."min_order_price" AS "min_price", "restaurant_restaurant"."delivery_charge" AS "delivery_charge", COUNT("restaurant_restauranttimeline_like"."user_id") AS "like_count" FROM "restaurant_restauranttimeline" INNER JOIN "restaurant_restaurant" ON ("restaurant_restauranttimeline"."restaurant_id" =

[{'id': 12,
  'created_time': datetime.datetime(2019, 6, 11, 10, 24, 5, 668631, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 11, 10, 24, 5, 668653, tzinfo=<UTC>),
  'restaurant_id': 8,
  'status': 1,
  'changed_data': '',
  'changed_img': '',
  'prior_info': '',
  'post_info': '',
  'timeline_type': 5,
  'changed_field': '',
  'restaurant_title': '교촌치킨 한양대점',
  'restaurant_name': '교촌치킨',
  'restaurant_img': 'restaurant/2019/05/31/kyochon.jpeg',
  'min_price': 14000,
  'delivery_charge': 2000,
  'like_count': 0},
 {'id': 11,
  'created_time': datetime.datetime(2019, 6, 11, 10, 22, 7, 230931, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 11, 10, 22, 7, 230954, tzinfo=<UTC>),
  'restaurant_id': 9,
  'status': 2,
  'changed_data': '이벤트',
  'changed_img': '',
  'prior_info': '',
  'post_info': '',
  'timeline_type': 2,
  'changed_field': 'event',
  'restaurant_title': '신전 떡볶이 한양대점',
  'restaurant_name': '신전 떡볶이',
  'restaurant_img': 'restaurant/2019/05/31/sinjeo

In [31]:
for timeline in timeline_values:
    print(timeline['created_time'])

2019-06-11 10:24:05.668631+00:00
2019-06-11 10:22:07.230931+00:00
2019-06-10 07:18:29.655826+00:00


In [32]:
Menu.objects.all()

(0.002) SELECT "menu_menu"."id", "menu_menu"."created_time", "menu_menu"."updated_time", "menu_menu"."restaurant_id", "menu_menu"."name", "menu_menu"."img", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."type", "menu_menu"."hit", "menu_menu"."score", "menu_menu"."is_recommended", "menu_menu"."is_group_purchase", "menu_menu"."min_quantity", "menu_menu"."max_quantity" FROM "menu_menu"  LIMIT 21; args=()


<QuerySet [<Menu: 고추바사삭>, <Menu: 갈비천왕>, <Menu: 허니멜로>, <Menu: 볼케이노>, <Menu: 통날개>, <Menu: 통다리>, <Menu: 짜장면>, <Menu: 짜장밥>, <Menu: 짬뽕>, <Menu: 짬뽕밥>, <Menu: 군만두>, <Menu: 깐풍기>, <Menu: 탕수육 소>, <Menu: 탕수육 대>, <Menu: 광어회 대>, <Menu: 광어회 중>, <Menu: 광어회 소>, <Menu: 모듬회>, <Menu: 물회>, <Menu: 회덮밥>, '...(remaining elements truncated)...']>

In [2]:
list(Menu.objects.all().values())

(0.001) SELECT "menu_menu"."id", "menu_menu"."created_time", "menu_menu"."updated_time", "menu_menu"."restaurant_id", "menu_menu"."name", "menu_menu"."img", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."type", "menu_menu"."hit", "menu_menu"."score", "menu_menu"."is_recommended", "menu_menu"."is_yosigy", "menu_menu"."is_set_menu" FROM "menu_menu"; args=()


[{'id': 1,
  'created_time': datetime.datetime(2019, 5, 2, 6, 8, 55, 691000, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 5, 2, 6, 8, 55, 720000, tzinfo=<UTC>),
  'restaurant_id': 1,
  'name': '고추바사삭',
  'img': 'menu/2019/05/01/고추바사삭.jpeg',
  'detail': '매콤 바삭한 치킨',
  'price': 18000,
  'type': '인기 메뉴',
  'hit': 0,
  'score': 0.0,
  'is_recommended': False,
  'is_yosigy': False,
  'is_set_menu': False},
 {'id': 2,
  'created_time': datetime.datetime(2019, 5, 2, 6, 8, 55, 691000, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 5, 2, 6, 8, 55, 720000, tzinfo=<UTC>),
  'restaurant_id': 1,
  'name': '갈비천왕',
  'img': 'menu/2019/05/01/갈비천왕.jpg',
  'detail': '갈비양념이 들어간 치킨',
  'price': 20000,
  'type': '양념류',
  'hit': 0,
  'score': 0.0,
  'is_recommended': False,
  'is_yosigy': False,
  'is_set_menu': False},
 {'id': 3,
  'created_time': datetime.datetime(2019, 5, 2, 6, 8, 55, 691000, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 5, 2, 7, 49, 57, 4400

In [23]:
from restaurant.models import Restaurant
restaurant_obj = Restaurant.objects.filter(id=9)
restaurant_menus = [menu['menu']for menu in list(restaurant_obj.values('menu'))]
restaurant_menus 

(0.001) SELECT "menu_menu"."id" FROM "restaurant_restaurant" LEFT OUTER JOIN "menu_menu" ON ("restaurant_restaurant"."id" = "menu_menu"."restaurant_id") WHERE "restaurant_restaurant"."id" = 9; args=(9,)


[36, 35, 34, 33]

In [22]:
menus 

[36, 35, 34, 33]

In [24]:
dict(menus)

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [26]:
menu_list =[{"id": "36", "discounted_price": "17800"}, {"id": "35", "discounted_price": "17900"}]

In [29]:
for menu in menu_list:
    menu_ids = 

SyntaxError: invalid syntax (<ipython-input-29-b15cada210ab>, line 1)

In [31]:
res=Restaurant.objects.get(pk=9)

(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."store_owner_id", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "restaurant_restaurant"."delivery_charge", "restaurant_restaurant"."info", "restaurant_restaurant"."event", "restaurant_restaurant"."event_img", "restaurant_restaurant"."type", "restaurant_restaurant"."estimated_delivery_time", "restaurant_restaurant"."operation_start_hour", "restaurant_restaurant"."operation_end_hour", "restaurant_restaurant"."is_yosigy" FROM "restaurant_restaurant" WHERE "restaurant_restaurant"."id" = 9; args=(9,)


In [46]:
menu_list =list(res.menu_set.all().values('id','price'))

(0.001) SELECT "menu_menu"."id", "menu_menu"."price" FROM "menu_menu" WHERE "menu_menu"."restaurant_id" = 9; args=(9,)


In [41]:
user = User.objects.get(username='admin')
res =Restaurant.objects.get(id=9, store_owner=user)

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)
(0.001) SELECT "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."store_owner_id", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."min_order_price", "restaurant_restaurant"."order_way", "restaurant_restaurant"."origin", "r

In [49]:
menu_list[0]['id']

36

In [51]:
menu_list[0]['price']-200

7800

In [62]:
user = User.objects.get(username='admin')
subscribed_restaurants = user.subscribed_restaurants.values('id')


(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)


In [74]:
subscribed_restaurants_ids = [restaurant['id'] for restaurant in subscribed_restaurants]
timeline_qs =(RestaurantTimeline.objects
            .select_related('restaurant', 'yosigy')
            .filter(restaurant_id__in=subscribed_restaurants_ids).order_by('-created_time'))

In [76]:
timeline_values = (
        timeline_qs
        .annotate(
            restaurant_title=F('restaurant__title'),
            restaurant_name=F('restaurant__name'),
            restaurant_img=F('restaurant__img'),
            min_price=F('restaurant__min_order_price'),
            delivery_charge=F('restaurant__delivery_charge'),
            like_count=Count('like')
        ).values())

In [77]:
list(timeline_values)

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_data", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info", "restaurant_restauranttimeline"."timeline_type", "restaurant_restauranttimeline"."changed_field", "restaurant_restauranttimeline"."yosigy_id", "restaurant_restaurant"."title" AS "restaurant_title", "restaurant_restaurant"."name" AS "restaurant_name", "restaurant_restaurant"."img" AS "restaurant_img", "restaurant_restaurant"."min_order_price" AS "min_price", "restaurant_restaurant"."delivery_charge" AS "delivery_charge", COUNT("restaurant_restauranttimeline_like"."user_id") AS "like_count" FROM "restaurant_restauranttimeline" INNER JOIN "restaurant_restaurant" ON ("res

[{'id': 56,
  'created_time': datetime.datetime(2019, 6, 18, 3, 25, 8, 682189, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 18, 3, 25, 8, 682208, tzinfo=<UTC>),
  'restaurant_id': 5,
  'status': 1,
  'changed_data': '',
  'changed_img': '',
  'prior_info': '',
  'post_info': '',
  'timeline_type': 5,
  'changed_field': '요식이 식권 이벤트',
  'yosigy_id': 54,
  'restaurant_title': '홍콩반점-서초점',
  'restaurant_name': '홍콩반점',
  'restaurant_img': 'restaurant/2019/05/21/홍콩반점.jpeg',
  'min_price': 10000,
  'delivery_charge': 2000,
  'like_count': 0},
 {'id': 13,
  'created_time': datetime.datetime(2019, 6, 14, 2, 31, 31, 416737, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 14, 2, 33, 8, 655835, tzinfo=<UTC>),
  'restaurant_id': 1,
  'status': 2,
  'changed_data': '배달 이벤트 시작합니다!! 두둥!!',
  'changed_img': 'timeline/2019/06/14/1559264402767.jpg',
  'prior_info': '',
  'post_info': '',
  'timeline_type': 2,
  'changed_field': 'owner',
  'yosigy_id': None,
  'restaurant

In [10]:
user = User.objects.get(username='admin')


(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)


In [14]:
from django.db.models import Count
list(user.ticketpayment_set.select_related('yosigy_ticket').all().annotate(ticket_num = Count('yosigy_ticket')).order_by('-created_time').values())

(0.001) SELECT "yosigy_ticketpayment"."id", "yosigy_ticketpayment"."created_time", "yosigy_ticketpayment"."updated_time", "yosigy_ticketpayment"."user_id", "yosigy_ticketpayment"."total_price", COUNT("yosigy_ticketpayment_yosigy_ticket"."yosigyticket_id") AS "ticket_num" FROM "yosigy_ticketpayment" LEFT OUTER JOIN "yosigy_ticketpayment_yosigy_ticket" ON ("yosigy_ticketpayment"."id" = "yosigy_ticketpayment_yosigy_ticket"."ticketpayment_id") WHERE "yosigy_ticketpayment"."user_id" = 1 GROUP BY "yosigy_ticketpayment"."id" ORDER BY "yosigy_ticketpayment"."created_time" DESC; args=(1,)


[{'id': 2,
  'created_time': datetime.datetime(2019, 6, 18, 7, 8, 42, 257140, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 18, 8, 21, 41, 733140, tzinfo=<UTC>),
  'user_id': 1,
  'total_price': 0,
  'ticket_num': 2}]

In [1]:
user = User.objects.get(username='admin')


(0.002) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)


In [4]:
user.ticketpayment_set.select_related('yosigy_ticket').all()

FieldError: Invalid field name(s) given in select_related: 'yosigy_ticket'. Choices are: user

In [5]:
from yosigy.models import TicketPayment

In [10]:
(user.ticketpayment_set.select_related('yosigy_ticket').all()
                )

FieldError: Invalid field name(s) given in select_related: 'yosigy_ticket'. Choices are: user

In [26]:
ticket_payment=user.ticketpayment_set.filter(id=2).values('yosigy_ticket__menu__name','yosigy_ticket__yosigy_menu__discounted_price')

In [27]:
list(ticket_payment)
    

(0.002) SELECT "menu_menu"."name", "yosigy_yosigymenu"."discounted_price" FROM "yosigy_ticketpayment" LEFT OUTER JOIN "yosigy_ticketpayment_yosigy_ticket" ON ("yosigy_ticketpayment"."id" = "yosigy_ticketpayment_yosigy_ticket"."ticketpayment_id") LEFT OUTER JOIN "yosigy_yosigyticket" ON ("yosigy_ticketpayment_yosigy_ticket"."yosigyticket_id" = "yosigy_yosigyticket"."id") LEFT OUTER JOIN "menu_menu" ON ("yosigy_yosigyticket"."menu_id" = "menu_menu"."id") LEFT OUTER JOIN "yosigy_yosigymenu" ON ("yosigy_yosigyticket"."yosigy_menu_id" = "yosigy_yosigymenu"."id") WHERE ("yosigy_ticketpayment"."user_id" = 1 AND "yosigy_ticketpayment"."id" = 2); args=(1, 2)


[{'yosigy_ticket__menu__name': '탕수육 소',
  'yosigy_ticket__yosigy_menu__discounted_price': 10000},
 {'yosigy_ticket__menu__name': '군만두',
  'yosigy_ticket__yosigy_menu__discounted_price': 500}]

In [29]:
ticket_payment = user.ticketpayment_set.filter(id=2)

In [33]:
list(ticket_payment.values())[0]

(0.001) SELECT "yosigy_ticketpayment"."id", "yosigy_ticketpayment"."created_time", "yosigy_ticketpayment"."updated_time", "yosigy_ticketpayment"."user_id", "yosigy_ticketpayment"."total_price" FROM "yosigy_ticketpayment" WHERE ("yosigy_ticketpayment"."user_id" = 1 AND "yosigy_ticketpayment"."id" = 2); args=(1, 2)


{'id': 2,
 'created_time': datetime.datetime(2019, 6, 18, 7, 8, 42, 257140, tzinfo=<UTC>),
 'updated_time': datetime.datetime(2019, 6, 18, 9, 8, 28, 534543, tzinfo=<UTC>),
 'user_id': 1,
 'total_price': 0}

In [5]:
user = User.objects.get(username='admin')
user_ticket_payment = (
            user.ticketpayment_set.select_related('restaurant').all()
                .annotate(
                    restaurant_name=F('restaurant__title'),
                    ticket_count=Count('yosigy_ticket'))
                .order_by('-created_time').values())

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)


In [6]:
list(user_ticket_payment)


(0.001) SELECT "yosigy_ticketpayment"."id", "yosigy_ticketpayment"."created_time", "yosigy_ticketpayment"."updated_time", "yosigy_ticketpayment"."user_id", "yosigy_ticketpayment"."restaurant_id", "yosigy_ticketpayment"."total_price", "restaurant_restaurant"."title" AS "restaurant_name", COUNT("yosigy_ticketpayment_yosigy_ticket"."yosigyticket_id") AS "ticket_count" FROM "yosigy_ticketpayment" INNER JOIN "restaurant_restaurant" ON ("yosigy_ticketpayment"."restaurant_id" = "restaurant_restaurant"."id") LEFT OUTER JOIN "yosigy_ticketpayment_yosigy_ticket" ON ("yosigy_ticketpayment"."id" = "yosigy_ticketpayment_yosigy_ticket"."ticketpayment_id") WHERE "yosigy_ticketpayment"."user_id" = 1 GROUP BY "yosigy_ticketpayment"."id", "restaurant_restaurant"."title" ORDER BY "yosigy_ticketpayment"."created_time" DESC; args=(1,)


[{'id': 2,
  'created_time': datetime.datetime(2019, 6, 18, 7, 8, 42, 257140, tzinfo=<UTC>),
  'updated_time': datetime.datetime(2019, 6, 18, 9, 8, 28, 534543, tzinfo=<UTC>),
  'user_id': 1,
  'restaurant_id': 5,
  'total_price': 0,
  'restaurant_name': '홍콩반점-서초점',
  'ticket_count': 3}]

In [40]:
Menu.objects.create(
            restaurant_id=2,
            name='치즈피자',
            detail='치즈가 듬뿍',
            price='12000',
            type='인기메뉴'
        )

(0.008) INSERT INTO "menu_menu" ("created_time", "updated_time", "restaurant_id", "name", "img", "detail", "price", "type", "hit", "score", "is_recommended", "is_yosigy", "is_set_menu") VALUES ('2019-06-20T01:52:48.033359+00:00'::timestamptz, '2019-06-20T01:52:48.033398+00:00'::timestamptz, 2, '치즈피자', '', '치즈가 듬뿍', 12000, '인기메뉴', 0, 0.0, false, false, false) RETURNING "menu_menu"."id"; args=(datetime.datetime(2019, 6, 20, 1, 52, 48, 33359, tzinfo=<UTC>), datetime.datetime(2019, 6, 20, 1, 52, 48, 33398, tzinfo=<UTC>), 2, '치즈피자', '', '치즈가 듬뿍', 12000, '인기메뉴', 0, 0.0, False, False, False)


<Menu: 치즈피자>

In [7]:
type(user_ticket_payment)

django.db.models.query.QuerySet

In [10]:
ticket_payment = TicketPayment.objects.filter(user=user, id=2)
tickets = (
            ticket_payment
            .select_related('restaurant')
            .prefetch_related('yosigy_ticket')
            .annotate(
                restaurant_name=F('restaurant__title'),
                yosigy_ticket_id=F('yosigy_ticket'),
                ticket_menu_id=F('yosigy_ticket__menu'),
                menu_id=F('yosigy_ticket__menu'),
                menu_price=F('yosigy_ticket__menu__price'),
                ticket_menu=F('yosigy_ticket__menu__name'),
                ticket_menu_price=F('yosigy_ticket__yosigy_menu__discounted_price'),
                ticket_expire_time=F('yosigy_ticket__expire_time'),
            
            ).order_by('ticket_menu_id')
            .values(
                'id', 'restaurant_id', 'restaurant_name', 'total_price', 'created_time',
                'yosigy_ticket_id', 'ticket_menu', 'menu_id', 'menu_price', 'ticket_menu_price', 'ticket_expire_time'))

In [11]:
list(tickets)

(0.006) SELECT "yosigy_ticketpayment"."id", "yosigy_ticketpayment"."restaurant_id", "yosigy_ticketpayment"."total_price", "yosigy_ticketpayment"."created_time", "restaurant_restaurant"."title" AS "restaurant_name", "yosigy_ticketpayment_yosigy_ticket"."yosigyticket_id" AS "yosigy_ticket_id", "yosigy_yosigyticket"."menu_id" AS "menu_id", "menu_menu"."price" AS "menu_price", "menu_menu"."name" AS "ticket_menu", "yosigy_yosigymenu"."discounted_price" AS "ticket_menu_price", "yosigy_yosigyticket"."expire_time" AS "ticket_expire_time" FROM "yosigy_ticketpayment" INNER JOIN "restaurant_restaurant" ON ("yosigy_ticketpayment"."restaurant_id" = "restaurant_restaurant"."id") LEFT OUTER JOIN "yosigy_ticketpayment_yosigy_ticket" ON ("yosigy_ticketpayment"."id" = "yosigy_ticketpayment_yosigy_ticket"."ticketpayment_id") LEFT OUTER JOIN "yosigy_yosigyticket" ON ("yosigy_ticketpayment_yosigy_ticket"."yosigyticket_id" = "yosigy_yosigyticket"."id") LEFT OUTER JOIN "menu_menu" ON ("yosigy_yosigyticket"."

[{'id': 2,
  'restaurant_id': 5,
  'total_price': 0,
  'created_time': datetime.datetime(2019, 6, 18, 7, 8, 42, 257140, tzinfo=<UTC>),
  'restaurant_name': '홍콩반점-서초점',
  'yosigy_ticket_id': 2,
  'menu_id': 11,
  'menu_price': 8000,
  'ticket_menu': '군만두',
  'ticket_menu_price': 500,
  'ticket_expire_time': datetime.datetime(2019, 6, 18, 8, 21, 21, tzinfo=<UTC>)},
 {'id': 2,
  'restaurant_id': 5,
  'total_price': 0,
  'created_time': datetime.datetime(2019, 6, 18, 7, 8, 42, 257140, tzinfo=<UTC>),
  'restaurant_name': '홍콩반점-서초점',
  'yosigy_ticket_id': 3,
  'menu_id': 11,
  'menu_price': 8000,
  'ticket_menu': '군만두',
  'ticket_menu_price': 500,
  'ticket_expire_time': datetime.datetime(2019, 6, 18, 9, 6, 59, tzinfo=<UTC>)},
 {'id': 2,
  'restaurant_id': 5,
  'total_price': 0,
  'created_time': datetime.datetime(2019, 6, 18, 7, 8, 42, 257140, tzinfo=<UTC>),
  'restaurant_name': '홍콩반점-서초점',
  'yosigy_ticket_id': 1,
  'menu_id': 13,
  'menu_price': 13000,
  'ticket_menu': '탕수육 소',
  'ticket_

In [19]:
 tickets = (
            ticket_payment
            .select_related('restaurant')
            .annotate(
                restaurant_name=F('restaurant__title'),
                yosigy_ticket_id=F('yosigy_ticket'),
                ticket_menu_id=F('yosigy_ticket__menu'),
                menu_id=F('yosigy_ticket__menu'),
                menu_price=F('yosigy_ticket__menu__price'),
                ticket_menu=F('yosigy_ticket__menu__name'),
                ticket_menu_price=F('yosigy_ticket__yosigy_menu__discounted_price'),
                ticket_expire_time=F('yosigy_ticket__expire_time')
            ).order_by('ticket_menu_id')
 )
import json
json.dumps(list(tickets))
            

(0.002) SELECT "yosigy_ticketpayment"."id", "yosigy_ticketpayment"."created_time", "yosigy_ticketpayment"."updated_time", "yosigy_ticketpayment"."user_id", "yosigy_ticketpayment"."restaurant_id", "yosigy_ticketpayment"."total_price", "restaurant_restaurant"."title" AS "restaurant_name", "yosigy_ticketpayment_yosigy_ticket"."yosigyticket_id" AS "yosigy_ticket_id", "yosigy_yosigyticket"."menu_id" AS "ticket_menu_id", "yosigy_yosigyticket"."menu_id" AS "menu_id", "menu_menu"."price" AS "menu_price", "menu_menu"."name" AS "ticket_menu", "yosigy_yosigymenu"."discounted_price" AS "ticket_menu_price", "yosigy_yosigyticket"."expire_time" AS "ticket_expire_time", "restaurant_restaurant"."id", "restaurant_restaurant"."created_time", "restaurant_restaurant"."updated_time", "restaurant_restaurant"."name", "restaurant_restaurant"."owner", "restaurant_restaurant"."store_owner_id", "restaurant_restaurant"."title", "restaurant_restaurant"."img", "restaurant_restaurant"."tel", "restaurant_restaurant"."

TypeError: Object of type TicketPayment is not JSON serializable

In [20]:
list(tickets)

[<TicketPayment: 2-0원>, <TicketPayment: 2-0원>, <TicketPayment: 2-0원>]

In [22]:
tickets = (
            ticket_payment
                .select_related('restaurant')
                .prefetch_related(Prefetch('yosigy_ticket',queryset=YosigyTicket.objects.select_related('menu'))))

In [33]:
ticket_payment=list(tickets)[0]
type(ticket_payment)

yosigy.models.TicketPayment

In [30]:
ticket_payment=list(tickets)[0]
ticket_payment[0].yosigy_ticket.all()


wanted_asdf=
{'id':ticket_payment.id,
 'restaurant_id':ticket_payment.restaurant.id,
 'restaurant_name':ticket_payment.restaurant.title,
 'total_price':ticket_payment.total_price,
 'created_time': ticket_payment.created_time.strftime('%Y-%m-%d %H:%M'),
 'yosigy_tickets':[
     {
        'yosigy_ticket_id': y_ticket.id,
        'ticket_menu': y_ticket.menu.name,
        'menu_id': y_ticket.menu.id,
        'menu_price': y_ticket.menu.price, 
        'ticket_menu_price': y_ticket.yosigy_menu.discounted_price,
        'ticket_expire_time': y_ticket.exprie_time.strftime('%Y-%m-%d %H:%M') 
     }
     for y_ticket in ticket_payment.yogiyo_ticket.all()
 ]
}
    

SyntaxError: invalid syntax (<ipython-input-30-26f51b3571e2>, line 5)

In [42]:
from order.models import Order
order= Order.objects.all().order_by('created_time')
order[0]

(0.001) SELECT "order_order"."created_time", "order_order"."updated_time", "order_order"."id", "order_order"."user_id", "order_order"."cart_id", "order_order"."restaurant_id", "order_order"."total_price", "order_order"."address", "order_order"."address_detail", "order_order"."status", "order_order"."delivery_status", "order_order"."weather", "order_order"."phone_num", "order_order"."payment_status", "order_order"."gift_coupon_id" FROM "order_order" ORDER BY "order_order"."created_time" ASC  LIMIT 1; args=()
(0.000) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accou

<Order: mhshimtest - 굽네치킨 - weather : 1>

In [39]:
order[1].yosigy_ticket.all()



(0.001) SELECT "order_order"."created_time", "order_order"."updated_time", "order_order"."id", "order_order"."user_id", "order_order"."cart_id", "order_order"."restaurant_id", "order_order"."total_price", "order_order"."address", "order_order"."address_detail", "order_order"."status", "order_order"."delivery_status", "order_order"."weather", "order_order"."phone_num", "order_order"."payment_status", "order_order"."gift_coupon_id" FROM "order_order"  LIMIT 1 OFFSET 1; args=()
(0.001) SELECT "yosigy_yosigyticket"."id", "yosigy_yosigyticket"."created_time", "yosigy_yosigyticket"."updated_time", "yosigy_yosigyticket"."user_id", "yosigy_yosigyticket"."yosigy_menu_id", "yosigy_yosigyticket"."menu_id", "yosigy_yosigyticket"."expire_time", "yosigy_yosigyticket"."status" FROM "yosigy_yosigyticket" INNER JOIN "order_order_yosigy_ticket" ON ("yosigy_yosigyticket"."id" = "order_order_yosigy_ticket"."yosigyticket_id") WHERE "order_order_yosigy_ticket"."order_id" = '058518bf-7234-43a1-b653-9ceefa276

<QuerySet []>

In [43]:
user_ticket_payment = (
            user.ticketpayment_set.select_related('restaurant').all()
                .annotate(
                    restaurant_name=F('restaurant__title'),
                    restaurant_img=F('restaurant__img'),
                    ticket_count=Count('yosigy_ticket'),
                    payment_time=F('created_time'))
                .extra(select={'datestr': "DATE_FORMAT(payment_time, '%Y-%m-%d')"})
            .order_by('-payment_time').values())

In [44]:
user_ticket_payment

(0.000) None; args=(1,)


IndexError: tuple index out of range

In [4]:
from restaurant.models import Restaurant

prior_restaurant_obj = Restaurant.objects.filter(pk=2).values()

In [10]:
list(prior_restaurant_obj)[0]['store_owner__username']

KeyError: 'store_owner__username'

In [9]:
user = User.objects.get(username='admin')
restaurant_timeline_id=56

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = 'admin'; args=('admin',)


In [10]:
restaurant_timeline = RestaurantTimeline.objects.filter(id=restaurant_timeline_id)

In [21]:
result=restaurant_timeline[0].like.add(user)

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_data", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info", "restaurant_restauranttimeline"."timeline_type", "restaurant_restauranttimeline"."changed_field", "restaurant_restauranttimeline"."yosigy_id" FROM "restaurant_restauranttimeline" WHERE "restaurant_restauranttimeline"."id" = 56  LIMIT 1; args=(56,)
(0.000) SELECT "restaurant_restauranttimeline_like"."user_id" FROM "restaurant_restauranttimeline_like" WHERE ("restaurant_restauranttimeline_like"."restauranttimeline_id" = 56 AND "restaurant_restauranttimeline_like"."user_id" IN (1)); args=(56, 1)


In [22]:
print(result)

None


In [23]:
timeline_like = restaurant_timeline[0].like

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_data", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info", "restaurant_restauranttimeline"."timeline_type", "restaurant_restauranttimeline"."changed_field", "restaurant_restauranttimeline"."yosigy_id" FROM "restaurant_restauranttimeline" WHERE "restaurant_restauranttimeline"."id" = 56  LIMIT 1; args=(56,)


In [50]:
timeline_like.filter()

<django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager at 0x10fee9f28>

In [34]:
user.restaurant_timeline_like

AttributeError: 'User' object has no attribute 'restaurant_timeline_like'

In [52]:
subscribed_restaurants = user.subscribed_restaurants.values('id')
        


subscribed_restaurants_ids = [restaurant['id'] for restaurant in subscribed_restaurants]

timeline_qs = (
    RestaurantTimeline.objects
    .select_related('restaurant', 'yosigy')
    .filter(restaurant_id__in=subscribed_restaurants_ids).order_by('-created_time'))

timeline_values = (
    timeline_qs
    .annotate(
        restaurant_title=F('restaurant__title'),
        restaurant_name=F('restaurant__name'),
        restaurant_img=F('restaurant__img'),
        min_price=F('restaurant__min_order_price'),
        delivery_charge=F('restaurant__delivery_charge'),
        like_count=Count('like'),
        yosigy_notice=F('yosigy__notice'),
        yosigy_deadline=F('yosigy__deadline')
    ))

(0.001) SELECT "restaurant_restaurant"."id" FROM "restaurant_restaurant" INNER JOIN "accounts_user_subscribed_restaurants" ON ("restaurant_restaurant"."id" = "accounts_user_subscribed_restaurants"."restaurant_id") WHERE "accounts_user_subscribed_restaurants"."user_id" = 1; args=(1,)


In [48]:
timeline_list = [
    {'restaurant_id': timeline['restaurant_id'],
     'restaurant_timeline_id': timeline['id'],
     'restaurant_name': timeline['restaurant_name'],
     'restaurant_title': timeline['restaurant_title'],
     'restaurant_img': timeline['restaurant_img'],
     'min_price': timeline['min_price'],
     'delivery_charge':timeline['delivery_charge'],
     'created_time':timeline['created_time'],
     'updated_time':timeline['updated_time'],
     'post_img': timeline['changed_img'],
     'prior_info': timeline['prior_info'],
     'post_info': (timeline['yosigy_notice']
                   if timeline['yosigy_id'] is not None else timeline['post_info']+timeline['changed_data']),
     'timeline_type': timeline['timeline_type'],
     'changed_item': timeline['changed_field'],
     'like': timeline['like_flag'],
     'like_count': timeline['like_count'],
     'yosigy_id': timeline['yosigy_id'],
     'yosidy_deadline':timeline['yosigy_deadline']}
    for timeline in timeline_values]

In [51]:
timeline_list[0]

{'restaurant_id': 1,
 'restaurant_timeline_id': 58,
 'restaurant_name': '굽네치킨',
 'restaurant_title': '굽네치킨-서초점',
 'restaurant_img': 'restaurant/2019/05/05/굽네치킨.jpg',
 'min_price': 17000,
 'delivery_charge': 2000,
 'created_time': datetime.datetime(2019, 6, 21, 3, 23, 8, 800589, tzinfo=<UTC>),
 'updated_time': datetime.datetime(2019, 6, 21, 3, 23, 8, 800611, tzinfo=<UTC>),
 'post_img': '',
 'prior_info': '',
 'post_info': '',
 'timeline_type': 5,
 'changed_item': '요식이 식권 이벤트',
 'like': 1,
 'like_count': 1,
 'yosigy_id': 55,
 'yosidy_deadline': datetime.date(2019, 7, 1)}

In [54]:
timeline=timeline_values[0]

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_data", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info", "restaurant_restauranttimeline"."timeline_type", "restaurant_restauranttimeline"."changed_field", "restaurant_restauranttimeline"."yosigy_id", "restaurant_restaurant"."title" AS "restaurant_title", "restaurant_restaurant"."name" AS "restaurant_name", "restaurant_restaurant"."img" AS "restaurant_img", "restaurant_restaurant"."min_order_price" AS "min_price", "restaurant_restaurant"."delivery_charge" AS "delivery_charge", COUNT("restaurant_restauranttimeline_like"."user_id") AS "like_count", "yosigy_yosigy"."notice" AS "yosigy_notice", "yosigy_yosigy"."deadline" AS "yos

In [55]:
timeline.like

<django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager at 0x10ff1e400>

In [73]:
timeline.like.filter(user__in=user)


ValueError: Cannot query "admin": Must be "UserGiftCoupon" instance.

In [89]:
list(user.restauranttimeline_set.values('id')).values()

(0.001) SELECT "restaurant_restauranttimeline"."id" FROM "restaurant_restauranttimeline" INNER JOIN "restaurant_restauranttimeline_like" ON ("restaurant_restauranttimeline"."id" = "restaurant_restauranttimeline_like"."restauranttimeline_id") WHERE "restaurant_restauranttimeline_like"."user_id" = 1; args=(1,)


AttributeError: 'list' object has no attribute 'values'

In [98]:
restaurant_timeline = RestaurantTimeline.objects.filter(id=55)

In [99]:
restaurant_timeline[0]

(0.001) SELECT "restaurant_restauranttimeline"."id", "restaurant_restauranttimeline"."created_time", "restaurant_restauranttimeline"."updated_time", "restaurant_restauranttimeline"."restaurant_id", "restaurant_restauranttimeline"."status", "restaurant_restauranttimeline"."changed_data", "restaurant_restauranttimeline"."changed_img", "restaurant_restauranttimeline"."prior_info", "restaurant_restauranttimeline"."post_info", "restaurant_restauranttimeline"."timeline_type", "restaurant_restauranttimeline"."changed_field", "restaurant_restauranttimeline"."yosigy_id" FROM "restaurant_restauranttimeline" WHERE "restaurant_restauranttimeline"."id" = 55  LIMIT 1; args=(55,)


IndexError: list index out of range

In [106]:
from django.db.models import Count, F
timeline=RestaurantTimeline.objects.annotate(comment_id=F('restauranttimelinecomment')).get(id=58)

FieldError: Cannot resolve keyword 'restauranttimelinecomment' into field. Choices are: changed_data, changed_field, changed_img, created_time, id, like, post_info, prior_info, restaurant, restaurant_id, status, timeline_type, updated_time, yosigy, yosigy_id

In [108]:
timeline.like.count()

(0.001) SELECT COUNT(*) AS "__count" FROM "accounts_user" INNER JOIN "restaurant_restauranttimeline_like" ON ("accounts_user"."id" = "restaurant_restauranttimeline_like"."user_id") WHERE "restaurant_restauranttimeline_like"."restauranttimeline_id" = 58; args=(58,)


1

In [29]:
Menu.objects.prefetch_related('tastes').filter(
    Q(tastes=5)&Q(tastes=6)&Q(tastes=None) 
)
Menu.objects.prefetch_related('tastes').filter(tatest=5)##, queryset=(Menu.objects.filter(Q(tastes=5)&Q(tastes=6)))))

FieldError: Cannot resolve keyword 'tatest' into field. Choices are: cartitem, created_time, detail, grouppurchase, hit, id, img, is_recommended, is_set_menu, is_yosigy, like, menutimeline, name, price, restaurant, restaurant_id, score, tastes, type, updated_time, yosigymenu, yosigyticket

In [4]:
a = Menu.objects.filter(
    Q(tastes=5)
)

b = Menu.objects.filter(
    Q(tastes=6)
)

a & b

(0.002) SELECT "menu_menu"."id", "menu_menu"."created_time", "menu_menu"."updated_time", "menu_menu"."restaurant_id", "menu_menu"."name", "menu_menu"."img", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."type", "menu_menu"."hit", "menu_menu"."score", "menu_menu"."is_recommended", "menu_menu"."is_yosigy", "menu_menu"."is_set_menu" FROM "menu_menu" INNER JOIN "menu_menu_tastes" ON ("menu_menu"."id" = "menu_menu_tastes"."menu_id") LEFT OUTER JOIN "menu_menu_tastes" T4 ON ("menu_menu"."id" = T4."menu_id") WHERE ("menu_menu_tastes"."taste_id" = 5 AND T4."taste_id" = 6)  LIMIT 21; args=(5, 6)


<QuerySet [<Menu: 짬뽕>]>

In [8]:
Menu.objects.filter(
    Q(tastes=5)
).filter(Q(tastes=6))

(0.001) SELECT "menu_menu"."id", "menu_menu"."created_time", "menu_menu"."updated_time", "menu_menu"."restaurant_id", "menu_menu"."name", "menu_menu"."img", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."type", "menu_menu"."hit", "menu_menu"."score", "menu_menu"."is_recommended", "menu_menu"."is_yosigy", "menu_menu"."is_set_menu" FROM "menu_menu" INNER JOIN "menu_menu_tastes" ON ("menu_menu"."id" = "menu_menu_tastes"."menu_id") INNER JOIN "menu_menu_tastes" T4 ON ("menu_menu"."id" = T4."menu_id") WHERE ("menu_menu_tastes"."taste_id" = 5 AND T4."taste_id" = 6)  LIMIT 21; args=(5, 6)


<QuerySet [<Menu: 짬뽕>]>

In [42]:
Menu.objects.filter(Q(tastes=None)|(Q(tastes=5)&Q(tastes=6)))

(0.001) SELECT "menu_menu"."id", "menu_menu"."created_time", "menu_menu"."updated_time", "menu_menu"."restaurant_id", "menu_menu"."name", "menu_menu"."img", "menu_menu"."detail", "menu_menu"."price", "menu_menu"."type", "menu_menu"."hit", "menu_menu"."score", "menu_menu"."is_recommended", "menu_menu"."is_yosigy", "menu_menu"."is_set_menu" FROM "menu_menu" LEFT OUTER JOIN "menu_menu_tastes" ON ("menu_menu"."id" = "menu_menu_tastes"."menu_id") WHERE ("menu_menu_tastes"."taste_id" IS NULL OR ("menu_menu_tastes"."taste_id" = 5 AND "menu_menu_tastes"."taste_id" = 6))  LIMIT 21; args=(5, 6)


<QuerySet [<Menu: 갈비천왕>, <Menu: 탕수육 소>, <Menu: 고추바사삭>, <Menu: 탕수육 대>]>

In [59]:
cart=(Cart.objects
    .prefetch_related('order', 'cartitem')
    .filter(
        Q(order__delivery_status=2) &
        Q(order__status=1)&
        ~Q(order=None)
    ))

In [50]:
cart

(0.001) SELECT "cart_cart"."created_time", "cart_cart"."updated_time", "cart_cart"."id", "cart_cart"."user_id" FROM "cart_cart" INNER JOIN "order_order" ON ("cart_cart"."id" = "order_order"."cart_id") WHERE ("order_order"."delivery_status" = 2 AND "order_order"."status" = 1 AND NOT ("cart_cart"."id" IN (SELECT U0."id" FROM "cart_cart" U0 LEFT OUTER JOIN "order_order" U1 ON (U0."id" = U1."cart_id") WHERE (U1."id" IS NULL AND U0."id" = ("cart_cart"."id")))))  LIMIT 21; args=(2, 1)


AttributeError: Cannot find 'order' on Cart object, 'order' is an invalid parameter to prefetch_related()

In [56]:
from cart.models import Cart
Cart.objects.prefetch_related('order', 'cartitem')

(0.000) SELECT "cart_cart"."created_time", "cart_cart"."updated_time", "cart_cart"."id", "cart_cart"."user_id" FROM "cart_cart"  LIMIT 21; args=()


AttributeError: Cannot find 'order' on Cart object, 'order' is an invalid parameter to prefetch_related()

In [68]:
(Cart.objects
.filter(
    Q(order__delivery_status=2) &
    Q(order__status=1) &
    ~Q(order=None)
)
.distinct()
.annotate(
                menu_id=F('cartitem__menu__id'),
                img=F('cartitem__menu__img'),
                name=F('cartitem__menu__name'),
                detail=F('cartitem__menu__detail'),
                price=F('cartitem__menu__price'),
                restaurant_name=F('cartitem__menu__restaurant__name'),
                restaurant_min_order_price=F(
                    'cartitem__menu__restaurant__min_order_price'),
                restaurant_delivery_charge=F(
                    'cartitem__menu__restaurant__delivery_charge'),
            ).values(
                'menu_id',
                'img',
                'name',
                'detail',
                'price',
                'restaurant_name',
                'restaurant_min_order_price',
                'restaurant_delivery_charge',
            ))

(0.002) SELECT DISTINCT "cart_cartitem"."menu_id" AS "menu_id", "menu_menu"."img" AS "img", "menu_menu"."name" AS "name", "menu_menu"."detail" AS "detail", "menu_menu"."price" AS "price", "restaurant_restaurant"."name" AS "restaurant_name", "restaurant_restaurant"."min_order_price" AS "restaurant_min_order_price", "restaurant_restaurant"."delivery_charge" AS "restaurant_delivery_charge" FROM "cart_cart" INNER JOIN "order_order" ON ("cart_cart"."id" = "order_order"."cart_id") LEFT OUTER JOIN "cart_cartitem" ON ("cart_cart"."id" = "cart_cartitem"."cart_id") LEFT OUTER JOIN "menu_menu" ON ("cart_cartitem"."menu_id" = "menu_menu"."id") LEFT OUTER JOIN "restaurant_restaurant" ON ("menu_menu"."restaurant_id" = "restaurant_restaurant"."id") WHERE ("order_order"."delivery_status" = 2 AND "order_order"."status" = 1 AND NOT ("cart_cart"."id" IN (SELECT U0."id" FROM "cart_cart" U0 LEFT OUTER JOIN "order_order" U1 ON (U0."id" = U1."cart_id") WHERE (U1."id" IS NULL AND U0."id" = ("cart_cart"."id"))

<QuerySet [{'menu_id': 2, 'img': 'menu/2019/05/01/갈비천왕.jpg', 'name': '갈비천왕', 'detail': '갈비양념이 들어간 치킨', 'price': 20000, 'restaurant_name': '굽네치킨', 'restaurant_min_order_price': 17000, 'restaurant_delivery_charge': 2000}, {'menu_id': 15, 'img': 'menu/2019/05/29/광어회_대.jpg', 'name': '광어회 대', 'detail': '잘 먹고 잘 자란 커다란 양식 광어', 'price': 40000, 'restaurant_name': '척횟집', 'restaurant_min_order_price': 15000, 'restaurant_delivery_charge': 2000}, {'menu_id': 18, 'img': 'menu/2019/05/29/모듬회.jpeg', 'name': '모듬회', 'detail': '연어 광어 우럭 참치 등 다양한 회를 맛볼 수 있음', 'price': 45000, 'restaurant_name': '척횟집', 'restaurant_min_order_price': 15000, 'restaurant_delivery_charge': 2000}, {'menu_id': 21, 'img': 'menu/2019/05/29/회덮밥.jpg', 'name': '회덮밥', 'detail': '쫄깃한 회와 고슬한 밥', 'price': 8000, 'restaurant_name': '척횟집', 'restaurant_min_order_price': 15000, 'restaurant_delivery_charge': 2000}, {'menu_id': 23, 'img': 'menu/2019/05/31/jjajangmyun.jpg', 'name': '짜장면', 'detail': '맛있는 짜장면', 'price': 5000, 'r

In [73]:
import datetime
(Restaurant.objects.distinct().select_related('yosigy')
            .filter(
               Q(store_owner=User.objects.get(username='사장님')) & (Q(yosigy=None) | Q(yosigy__deadline__lte=datetime.datetime.now()))
            )
            .order_by(
                'name'
            ).values('id', 'name', 'title'))

(0.001) SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."phone", "accounts_user"."address", "accounts_user"."address_detail", "accounts_user"."user_type", "accounts_user"."deactivate_date" FROM "accounts_user" WHERE "accounts_user"."username" = '사장님'; args=('사장님',)
(0.001) SELECT DISTINCT "restaurant_restaurant"."id", "restaurant_restaurant"."name", "restaurant_restaurant"."title" FROM "restaurant_restaurant" LEFT OUTER JOIN "yosigy_yosigy" ON ("restaurant_restaurant"."id" = "yosigy_yosigy"."restaurant_id") WHERE ("restaurant_restaurant"."store_owner_id" = 14 AND ("yosigy_yosigy"."id" IS NULL OR "yosigy_yosigy"."deadline" <= '2019-07-04'::date)) ORDER BY "restaurant_restaurant"."name" ASC  LIMIT 21; args

<QuerySet [{'id': 7, 'name': '금룡', 'title': '금룡 한양대점'}]>

In [75]:
a=[1,3,'five',['list', 'of', 'lists']]


In [77]:
a[-1][2]

'lists'

In [85]:
a[:3]

[1, 3, 'five']

In [86]:
list_a=[1,3,4,5,2,10]

In [88]:
list_a.sort()

In [89]:
list_a

[1, 2, 3, 4, 5, 10]

In [91]:
list_a.extend([7,13])

In [92]:
list_a

[1, 2, 3, 4, 5, 10, 7, 13]

In [93]:
list_a.insert(2,3)

In [95]:
list_a.insert(0,0)

In [96]:
list_a

[0, 1, 2, 3, 3, 4, 5, 10, 7, 13]

In [97]:
t = (1, 2, 3)

In [98]:
t[:2]

(1, 2)

In [99]:
t2=(1,2,3,4,5,6)

In [101]:
t2+t

(1, 2, 3, 4, 5, 6, 1, 2, 3)

In [102]:
t*3

(1, 2, 3, 1, 2, 3, 1, 2, 3)

In [103]:
dict = {'key1': '영어', 'key2': 123}

In [107]:
dict.items()

dict_items([('key1', '영어'), ('key2', 123)])

In [108]:
'key1' in dict

True

In [111]:
'영어' in dict.values()

True

In [113]:
def square_numbers(nums):
    result = []
    for i in nums:
        result.append(i * i)
    return result

my_nums = square_numbers([1, 2, 3, 4, 5])

print( my_nums)

[1, 4, 9, 16, 25]


In [115]:
def square_numbers(nums):
    for i in nums:
        yield i * i

my_nums = square_numbers([1, 2, 3, 4, 5])

print (next(my_nums))

1


In [116]:
print (next(my_nums))

4


In [117]:
my_num = [x*x for x in [1,2,3,4,5]]

In [118]:
my_num

[1, 4, 9, 16, 25]

In [119]:
for i in my_num:
    print(i)

1
4
9
16
25


In [120]:
my_num=(x*x for x in [1,2,3,4,5])

In [121]:
my_num

<generator object <genexpr> at 0x1101d6c00>

In [122]:
next(my_num)

1

In [123]:
list(my_num)

[4, 9, 16, 25]

In [124]:
names = ['최용호', '지길정', '진영욱', '김세훈', '오세훈', '김민우']
majors = ['컴퓨터 공학', '국문학', '영문학', '수학', '정치']


In [125]:
import os
import psutil

ModuleNotFoundError: No module named 'psutil'

In [126]:
lambda x: x+1

<function __main__.<lambda>(x)>

In [127]:
print(lambda x: x+1)

<function <lambda> at 0x110210598>


In [129]:
add= lambda x,y: x+y


In [130]:
add(1,3)

4

In [131]:
xs=list(range(10))

In [132]:
xs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [133]:
doubled=map(lambda x:x*2,xs)

In [134]:
list(doubled)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [135]:
squared=map(lambda x: (x+1)**2,xs)

In [136]:
list(squared)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [141]:
evens=filter(lambda x:x%2 is 0,xs)

In [139]:
list(evens)


[0, 2, 4, 6, 8]

In [148]:
xs=range(10)
doubled=(x*x for x in xs)

In [151]:
next(doubled)

4

In [152]:
line=','.join(str(x+1) for  x in xs)

In [153]:
line

'1,2,3,4,5,6,7,8,9,10'

In [163]:
def reduce(fn, xs, init=None):
    if init is None:
        return reduce(fn, xs[1:],xs[0])
    acc=init
    for x in xs:
        acc=fn(acc,x)
    return acc

In [156]:
xs=range(10)

In [164]:
total=reduce(lambda x,y:x+y, xs)

In [165]:
total

45

In [166]:
def reduce(fn,xs, init=None):
    if init is None:
        return reduce(fn, xs[1:], xs[0])
    acc=init
    for x in xs:
        acc=fn(acc, x)
    return acc

In [171]:
xs=range(1,10)

In [172]:
reduce(lambda x,y:x*y, xs)

362880

In [174]:
product=reduce(lambda x,y: x*y, xs)

In [175]:
factorial=lambda n: product(range(1,n+1))

In [176]:
factorial

<function __main__.<lambda>(n)>

In [177]:
product

362880

In [178]:
min_xs=reduce(lambda x, y :x if x<y else y, xs)

In [179]:
min_xs

1

In [180]:
max_xs=reduce(lambda x,y :x if x>y else y, xs)

In [181]:
max_xs

9

In [195]:
a=list('abcde')


In [200]:
a='hello world'

In [201]:
a[::-1]

'dlrow olleh'

In [230]:
list_a=[1,2,3,4,5]

In [214]:
list_a.reverse()

In [229]:
list_a

[5, 4]

In [228]:
list_a

[5, 4]

In [231]:
list_b=[list_a.pop() for x in list_a]

In [235]:
list_a=[1,2,3,4,5]
list_b=[list_a.pop() for i in range(len(list_a))]

In [236]:
list_b

[5, 4, 3, 2, 1]

In [237]:
class Node:
    def __init__(self, key, value):
        self.key=key
        self.value=value

In [238]:
class HashTable():
    def __init__(self, size=7):
        self._table=[[] for i in range(size)]
        self.size=size
        
    def set(self, key, value):
        new_node=Node(key, value)
        box, item, in_index =self.find(key)
        
        if item is None:
            box.append(new_node)
        else:
            item = new_node
            
    def get(self, key)

In [241]:
a=[i for i in range(10000)]
b=(i for i in range(10000))

In [243]:
import sys
sys.getsizeof(a)

87624

In [244]:
sys.getsizeof(b)

120

In [245]:
from celery import Celery

ModuleNotFoundError: No module named 'celery'

In [246]:
l=[9,5,5,7,3]

In [247]:
list(set(l))

[9, 3, 5, 7]